# Long Term Forecast Influenza using Web data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import time
import shutil
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm import tqdm
from datetime import datetime, timedelta
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt

from utils.preprocessing import normalize

In [3]:
# US, AU, KR influenza load

flu = {
    'us': pd.read_csv('./data/US_flu.csv'), 
    'au': pd.read_csv('./data/AU_flu.csv'),
    'kr': pd.read_csv('./data/KR_flu.csv') 
}

In [4]:
flu['us']

,weeks,ILI
0,10-01,19.8284
1,10-02,18.2749
2,10-03,19.2606
3,10-04,19.2495
4,10-05,20.8877
...,...,...
535,20-16,25.7453
536,20-17,20.2111
537,20-18,16.2086
538,20-19,13.5240


In [5]:
for nation in flu:
    print(f'{nation} flu data shape: {flu[nation].shape}')

us flu data shape: (540, 2)
au flu data shape: (540, 2)
kr flu data shape: (540, 2)


In [6]:
# US, AU, KR Google trends

trends = {
    'us': pd.read_csv('./data/US_trends.csv'), 
    'au': pd.read_csv('./data/AU_trends.csv'), 
    'kr': pd.read_csv('./data/KR_trends.csv') 
}

In [7]:
trends['us']

,weeks,influenza,H1N1 flu,swine flu,Flu,H1N1,seasonal flu,H1N1 virus,flu virus,H1N1 swine flu,...,polio epidemic,Afluria seasonal flu,hemolytic uremic syndrome HUS,croup,Jeff Duchin,inactivated vaccine,fevers aches,invasive pneumococcal disease,Sudden Acute Respiratory Syndrome,commonly infect humans
0,17-35,0,0,0,6,0,0,0,1,0,...,1,0,0,40,0,46,0,0,0,0
1,17-36,1,0,0,8,0,0,0,2,0,...,0,0,0,50,0,50,14,14,0,0
2,17-37,1,0,0,9,0,0,0,2,0,...,0,0,0,51,0,46,14,14,0,0
3,17-38,1,0,0,11,0,0,0,2,0,...,0,0,0,58,0,87,0,0,0,0
4,17-39,1,0,0,12,0,0,0,2,0,...,0,0,0,68,0,54,14,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,20-16,2,0,1,24,2,3,1,10,1,...,5,0,0,8,1,76,0,0,0,0
138,20-17,2,0,1,19,1,2,1,10,1,...,8,0,0,10,0,41,0,0,0,0
139,20-18,2,0,1,18,1,2,1,8,1,...,6,0,0,5,0,56,0,11,0,0
140,20-19,2,0,1,16,1,2,1,6,1,...,7,0,0,7,0,28,0,0,0,0


In [8]:
for nation in trends:
    print(f'{nation} trends data shape: {trends[nation].shape}')

us trends data shape: (142, 1001)
au trends data shape: (142, 966)
kr trends data shape: (142, 1001)


In [9]:
MIN_PRECIDENCE = 3 # 최소 시간 선행
MAX_PRECIDENCE = 15 # 최대 시간 선행
precidences = [i for i in range(MIN_PRECIDENCE, MAX_PRECIDENCE+1)]

corrs = {}

for nation in ['us', 'au', 'kr']:
    print('='*10, nation, '='*10)
    flu_data = flu[nation]
    trends_data = trends[nation]

    corrs_by_prcds = []
    keywords = trends_data.columns[1:] # weeks를 제외한 모든 열
    for keyword in tqdm(keywords):
        # correlation을 계산할 데이터 선택
        # trends의 weeks와 일치하는 flu데이터
        base = flu_data[flu_data.weeks.isin(trends_data.weeks)].ILI.reset_index(drop=True)
        target = trends_data[keyword] # 각 키워드의 trends 데이터
        
        corr_list = []
        for prcd in precidences:
            corr = base.corr(target.shift(prcd)) # precidence에 따라 trends를 shift/corr 계산
            corr_list.append(corr)
        corrs_by_prcds.append(corr_list)

    corrs_by_prcds = pd.DataFrame(corrs_by_prcds).T # 데이터 프레임 형식
    corrs_by_prcds.columns = keywords 
    corrs_by_prcds.insert(0, 'prcd', precidences) # 첫 번째 열에 precidence를 표시
    corrs[nation] = corrs_by_prcds

========== us ==========


100%|██████████| 1000/1000 [00:01<00:00, 736.89it/s]


========== au ==========


100%|██████████| 965/965 [00:01<00:00, 757.60it/s]


========== kr ==========


100%|██████████| 1000/1000 [00:01<00:00, 762.72it/s]


In [60]:
trends['us'].weeks[20:-10+1].values

array(['18-03', '18-04', '18-05', '18-06', '18-07', '18-08', '18-09',
       '18-10', '18-11', '18-12', '18-13', '18-14', '18-15', '18-16',
       '18-17', '18-18', '18-19', '18-20', '18-21', '18-22', '18-23',
       '18-24', '18-25', '18-26', '18-27', '18-28', '18-29', '18-30',
       '18-31', '18-32', '18-33', '18-34', '18-35', '18-36', '18-37',
       '18-38', '18-39', '18-40', '18-41', '18-42', '18-43', '18-44',
       '18-45', '18-46', '18-47', '18-48', '18-49', '18-50', '18-51',
       '18-52', '18-53', '19-01', '19-02', '19-03', '19-04', '19-05',
       '19-06', '19-07', '19-08', '19-09', '19-10', '19-11', '19-12',
       '19-13', '19-14', '19-15', '19-16', '19-17', '19-18', '19-19',
       '19-20', '19-21', '19-22', '19-23', '19-24', '19-25', '19-26',
       '19-27', '19-28', '19-29', '19-30', '19-31', '19-32', '19-33',
       '19-34', '19-35', '19-36', '19-37', '19-38', '19-39', '19-40',
       '19-41', '19-42', '19-43', '19-44', '19-45', '19-46', '19-47',
       '19-48', '19-

In [108]:
idx = flu['us'][flu['us'].weeks=='18-03'].index[0]

flu['us'][idx-5:idx]

,weeks,ILI
414,17-50,32.0050
415,17-51,44.3807
416,17-52,54.6361
417,18-01,53.5765
418,18-02,55.2437


In [47]:
[(keyword, coef, corrs['us'][keyword].argmax()+MIN_PRECIDENCE) for keyword, coef in corrs['us'].max().sort_values(ascending=False)[1:5].items()]

[('flu fever cough', 0.8748608947690277, 3),
 ('bronchitis', 0.8734438553088462, 3),
 ('colds', 0.8734000309965851, 6),
 ('Flu Pneumonia', 0.865196060775432, 3)]

In [58]:
def slice_data(flu_data, trends_data, corrs_data, 
               lookback=5, lookahead=5, topn=1, historic=7):
    flu_inputs = []
    flu_labels = []
    web_past_inputs = []
    web_future_inputs = []
    week_ranges = []
    
    keywords_rank = corrs_data.max().sort_values(ascending=False)
    keywords_rank = [(keyword, coef, corrs_data[keyword].argmax()+MIN_PRECIDENCE)
                     for keyword, coef in keywords_rank[1:topn+1].items()]
    
    available_weeks = trends_data['weeks'][MAX_PRECIDENCE+lookback:-lookahead+1].values
    for year_week in available_weeks:
        year, week = map(int, year_week.split('-'))
        
        flu_pos = flu_data[flu_data.weeks==f'{year}-{week:02}'].index[0]
        curr_flu = flu_data[flu_pos-lookback:flu_pos].ILI.values.reshape(-1, 1)
        curr_lbl = flu_data[flu_pos:flu_pos+lookahead].ILI.values
        
        for h in range(1, historic+1):
            flu_pos = flu_data[flu_data.weeks==f'{year-h}-{week:02}'].index[0]
            historic_flu = flu_data[flu_pos-lookback:flu_pos].ILI.values.reshape(-1, 1)
            curr_flu = np.hstack([curr_flu, historic_flu])
            
        web_past = None
        web_future = None
        for keyword, coef, prcd in keywords_rank:
            web_pos = trends_data[trends_data.weeks==f'{year}-{week:02}'].index[0]
            curr_web_past = trends_data[keyword][web_pos-lookback-prcd:web_pos-prcd].values.reshape(-1, 1)
            curr_web_future = np.random.uniform(1e-6, 1e-4, (lookahead, 1))
            curr_web_future[:prcd] = trends_data[keyword][web_pos-prcd:web_pos-prcd+lookahead].values.reshape(-1, 1)[:prcd]
            
            if web_past is None:
                web_past = curr_web_past
            else:
                web_past = np.hstack([web_past, curr_web_past])
            if web_future is None:
                web_future = curr_web_future
            else:
                web_future = np.hstack([web_future, curr_web_future])
            
        
        flu_inputs.append(curr_flu)
        flu_labels.append(curr_lbl)
        web_past_inputs.append(web_past)
        web_future_inputs.append(web_future)
    
    return flu_inputs, flu_labels, web_past_inputs, web_future_inputs

In [69]:
trends['us'][['weeks', 'flu fever cough', 'bronchitis', 'colds']][10:30]

,weeks,flu fever cough,bronchitis,colds
10,17-45,21,55,12
11,17-46,18,52,14
12,17-47,14,63,13
13,17-48,13,61,14
14,17-49,27,63,15
15,17-50,31,75,18
16,17-51,46,92,24
17,17-52,33,98,24
18,18-01,58,90,18
19,18-02,74,78,16


In [70]:
slice_data(flu['us'], trends['us'], corrs['us'], topn=10)[2][0]

array([[14, 63, 12, 16,  3,  1, 79,  7, 15, 40],
       [13, 61, 12, 23,  3,  1, 79,  8, 20, 43],
       [27, 63, 14, 24,  5,  1, 84,  6, 26, 41],
       [31, 75, 13, 35,  7,  3, 85, 15, 26, 46],
       [46, 92, 14, 45, 11,  4, 93, 10, 28, 55]])

In [74]:
slice_data(flu['us'], trends['us'], corrs['us'], topn=10, lookahead=7)[3][0]

array([[3.30000000e+01, 9.80000000e+01, 1.50000000e+01, 5.80000000e+01,
        1.10000000e+01, 4.00000000e+00, 1.00000000e+02, 1.70000000e+01,
        3.60000000e+01, 6.50000000e+01],
       [5.80000000e+01, 9.00000000e+01, 1.80000000e+01, 7.00000000e+01,
        1.50000000e+01, 6.00000000e+00, 9.00000000e+01, 1.80000000e+01,
        4.50000000e+01, 5.80000000e+01],
       [7.40000000e+01, 7.80000000e+01, 2.40000000e+01, 9.70000000e+01,
        1.80000000e+01, 5.00000000e+00, 8.60000000e+01, 1.70000000e+01,
        5.30000000e+01, 5.70000000e+01],
       [7.81622667e-06, 3.49647535e-06, 2.40000000e+01, 7.34253195e-05,
        7.35682964e-05, 7.72619588e-06, 7.73248802e-05, 9.18875564e-05,
        4.90000000e+01, 8.53496564e-05],
       [8.91204701e-05, 4.37305160e-05, 1.80000000e+01, 2.41379610e-05,
        7.00203722e-05, 4.97435409e-05, 3.95683328e-05, 8.05447478e-05,
        4.40000000e+01, 1.95526973e-05],
       [1.46297499e-05, 1.91828053e-05, 1.60000000e+01, 1.29969617e-05,
   

In [13]:
slice_data(flu['us'], trends['us'], None)[0][0]

array([[32.005 , 44.3807, 54.6361, 53.5765, 55.2437],
       [22.7731, 29.1465, 35.4919, 30.8102, 30.1875],
       [19.6571, 24.2021, 24.1083, 19.8859, 19.8985],
       [34.178 , 44.6555, 61.1497, 40.7898, 39.3762],
       [24.1674, 30.3071, 43.8724, 42.112 , 34.6492],
       [36.2552, 45.532 , 60.2598, 49.6368, 46.9072],
       [16.6429, 18.5274, 21.2413, 17.6709, 15.434 ],
       [25.5659, 33.8185, 34.3172, 26.9151, 29.1063]])

In [11]:
# lag load

US_trends_lag = pd.read_csv('./data/v2/US_trends_lag.csv', index_col=0)
AU_trends_lag = pd.read_csv('./data/v2/AU_trends_lag.csv', index_col=0)

KR_news_lag = pd.read_csv('./data/v2/KR_news_lag.csv', index_col=0)
KR_sns_lag = pd.read_csv('./data/v2/KR_sns_lag.csv', index_col=0)
KR_trends_lag = pd.read_csv('./data/v2/KR_trends_lag.csv', index_col=0)

In [ ]:
# max lag load

US_trends_max_lag = pd.read_csv('./data/v2/US_trends_max_lag.csv', index_col=0)
AU_trends_max_lag = pd.read_csv('./data/v2/AU_trends_max_lag.csv', index_col=0)

KR_news_max_lag = pd.read_csv('./data/v2/KR_news_max_lag.csv', index_col=0)
KR_sns_max_lag = pd.read_csv('./data/v2/KR_sns_max_lag.csv', index_col=0)
KR_trends_max_lag = pd.read_csv('./data/v2/KR_trends_max_lag.csv', index_col=0)

In [ ]:
# Model performance load

US_perform = pd.read_csv('./data/v2/US_perform4.csv', index_col=0)
AU_perform = pd.read_csv('./data/v2/AU_perform4.csv', index_col=0)
KR_perform = pd.read_csv('./data/v2/KR_perform4.csv', index_col=0)

In [ ]:
# Model prediction load

with open('./data/v2/US_predict4.pkl', 'rb') as f:
    US_predict = pickle.load(f)
with open('./data/v2/AU_predict4.pkl', 'rb') as f:
    AU_predict = pickle.load(f)
with open('./data/v2/KR_predict4.pkl', 'rb') as f:
    KR_predict = pickle.load(f)

In [ ]:
# perform_by_num_words

with open('./data/v2/perform_by_num_words.pkl', 'rb') as f:
    perform_by_num_words = pickle.load(f)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Attention, Dense, Concatenate, Average, TimeDistributed, LeakyReLU, Add, Bidirectional, Average
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import clear_session
import tensorflow.keras.backend as K

In [ ]:
class Multi_Encoder_v2(object):
    def __init__(self, mode='A', is_sg=False, **kwargs):
        
        self.look_back = kwargs['look_back']
        self.look_ahead = kwargs['look_ahead']
        self.past_year_num = kwargs['past_year_num'] if 'past_year_num' in kwargs else 1
        self.news_word_num = kwargs['news_word_num'] if 'news_word_num' in kwargs else 1
        self.sns_word_num = kwargs['sns_word_num'] if 'sns_word_num' in kwargs else 1
        self.trend_word_num = kwargs['trend_word_num'] if 'trend_word_num' in kwargs else 1
        self.layer_size = kwargs['layer_size'] if 'layer_size' in kwargs else 128
        self.is_sg = is_sg # single encoder
        self.mode = mode # N: news , S: SNS , T: Trend , A: N+S+T
        
        
        ENC_LSTM_UNIT = self.layer_size
        DENSE_UNIT = self.layer_size
        
        print(ENC_LSTM_UNIT, DENSE_UNIT)
        
        # Encoder1
        enc1_inputs = Input(shape=(self.look_back, self.past_year_num+1), name='enc1_inputs')
        
        enc1_lstm1 = LSTM(ENC_LSTM_UNIT, return_state=True, return_sequences=True, name='enc1_lstm1')
        enc1_outputs1, enc1_h1, enc1_c1 = enc1_lstm1(enc1_inputs)
        
        # Encoder2
        # news
        enc2_news_inputs = Input(shape=(self.look_back, self.news_word_num), name='enc2_news_inputs')
        
        enc2_news_lstm1 = LSTM(ENC_LSTM_UNIT, return_state=True, return_sequences=True, name='enc2_news_lstm1')
        enc2_news_outputs1, enc2_news_h1, enc2_news_c1 = enc2_news_lstm1(enc2_news_inputs)
        
        enc2_news_outputs = Concatenate(axis=-1, name='enc2_news_outputs_concat')([enc1_outputs1, enc2_news_outputs1])
        
        enc2_news_states = [
            [Concatenate(name='enc2_news_h1')([enc1_h1, enc2_news_h1]), Concatenate(name='enc2_news_c1')([enc1_c1, enc2_news_c1])],
        ]
        
        # sns
        enc2_sns_inputs = Input(shape=(self.look_back, self.sns_word_num), name='enc2_sns_inputs')
        
        enc2_sns_lstm1 = LSTM(ENC_LSTM_UNIT, return_state=True, return_sequences=True, name='enc2_sns_lstm1')
        enc2_sns_outputs1, enc2_sns_h1, enc2_sns_c1 = enc2_sns_lstm1(enc2_sns_inputs)
        
        enc2_sns_outputs = Concatenate(axis=-1, name='enc2_sns_outputs_concat')([enc1_outputs1, enc2_sns_outputs1])
        
        enc2_sns_states = [
            [Concatenate(name='enc2_sns_h1')([enc1_h1, enc2_sns_h1]), Concatenate(name='enc2_sns_c1')([enc1_c1, enc2_sns_c1])],
        ]
        
        # trend
        enc2_trend_inputs = Input(shape=(self.look_back, self.trend_word_num), name='enc2_trend_inputs')
        
        enc2_trend_lstm1 = LSTM(ENC_LSTM_UNIT, return_state=True, return_sequences=True, name='enc2_trend_lstm1')
        enc2_trend_outputs1, enc2_trend_h1, enc2_trend_c1 = enc2_trend_lstm1(enc2_trend_inputs)
        
        enc2_trend_outputs = Concatenate(axis=-1, name='enc2_trend_outputs_concat')([enc1_outputs1, enc2_trend_outputs1])
        
        enc2_trend_states = [
            [Concatenate(name='enc2_trend_h1')([enc1_h1, enc2_trend_h1]), Concatenate(name='enc2_trend_c1')([enc1_c1, enc2_trend_c1])],
        ]
        
        # Decoder
        if is_sg:
            enc2_news_states = [enc2_news_h1, enc2_news_c1]
            enc2_news_outputs = enc2_news_outputs1
            enc2_sns_states = [enc2_sns_h1, enc2_sns_c1]
            enc2_sns_outputs = enc2_sns_outputs1
            enc2_trend_states = [enc2_trend_h1, enc2_trend_c1]
            enc2_trend_outputs = enc2_trend_outputs1
            dec_lstm_unit = ENC_LSTM_UNIT
            
        else:
            enc2_news_states = enc2_news_states[0]
            enc2_sns_states = enc2_sns_states[0]
            enc2_trend_states = enc2_trend_states[0]
            dec_lstm_unit = ENC_LSTM_UNIT * 2
        
        
        # news
        dec_news_fn = self.news_word_num if is_sg else self.news_word_num+1
        dec_news_inputs = Input(shape=(None, dec_news_fn), name='dec_news_inputs')
        
        dec_news_lstm1 = LSTM(dec_lstm_unit, return_state=True, return_sequences=True, name='dec_news_lstm1')
        dec_news_outputs1, _, _ = dec_news_lstm1(dec_news_inputs, initial_state=enc2_news_states)
        
        dec_news_attn = Attention(name='dec_news_attn')([dec_news_outputs1, enc2_news_outputs])
        
        # sns
        dec_sns_fn = self.sns_word_num if is_sg else self.sns_word_num+1
        dec_sns_inputs = Input(shape=(None, dec_sns_fn), name='dec_sns_inputs')
        
        dec_sns_lstm1 = LSTM(dec_lstm_unit, return_state=True, return_sequences=True, name='dec_sns_lstm1')
        dec_sns_outputs1, _, _ = dec_sns_lstm1(dec_sns_inputs, initial_state=enc2_sns_states)
        
        dec_sns_attn = Attention(name='dec_sns_attn')([dec_sns_outputs1, enc2_sns_outputs])
        
        # trend
        dec_trend_fn = self.trend_word_num if is_sg else self.trend_word_num+1
        dec_trend_inputs = Input(shape=(None, dec_trend_fn), name='dec_trend_inputs')
        
        dec_trend_lstm1 = LSTM(dec_lstm_unit, return_state=True, return_sequences=True, name='dec_trend_lstm1')
        dec_trend_outputs1, _, _ = dec_trend_lstm1(dec_trend_inputs, initial_state=enc2_trend_states)
        
        dec_trend_attn = Attention(name='dec_trend_attn')([dec_trend_outputs1, enc2_trend_outputs])
        
        # concat & dense
        if mode == 'N':
            enc2_inputs = [enc2_news_inputs]
            enc2_outputs = enc2_news_outputs
            enc2_states = enc2_news_states
            dec_inputs = [dec_news_inputs]
            dec_attn_concat = dec_news_attn
            dec_lstm1 = dec_news_lstm1
        elif mode == 'S':
            enc2_inputs = [enc2_sns_inputs]
            enc2_outputs = enc2_sns_outputs
            enc2_states = enc2_sns_states
            dec_inputs = [dec_sns_inputs]
            dec_attn_concat = dec_sns_attn
            dec_lstm1 = dec_sns_lstm1
        elif mode == 'T':
            enc2_inputs = [enc2_trend_inputs]
            enc2_outputs = enc2_trend_outputs
            enc2_states = enc2_trend_states
            dec_inputs = [dec_trend_inputs]
            dec_attn_concat = dec_trend_attn
            dec_lstm1 = dec_trend_lstm1
        elif mode == 'NS':
            enc2_inputs = [enc2_news_inputs, enc2_sns_inputs]
            dec_inputs = [dec_news_inputs, dec_sns_inputs]
            dec_attn_concat = Concatenate(name='decattn_concat')([dec_news_attn, dec_sns_attn])
        elif mode == 'ST':
            enc2_inputs = [enc2_sns_inputs, enc2_trend_inputs]
            dec_inputs = [dec_sns_inputs, dec_trend_inputs]
            dec_attn_concat = Concatenate(name='decattn_concat')([dec_sns_attn, dec_trend_attn])
        elif mode == 'TN':
            enc2_inputs = [enc2_trend_inputs, enc2_news_inputs]
            dec_inputs = [dec_trend_inputs, dec_news_inputs]
            dec_attn_concat = Concatenate(name='decattn_concat')([dec_trend_attn, dec_news_attn])
        else:
            enc2_inputs = [enc2_news_inputs, enc2_sns_inputs, enc2_trend_inputs]
            enc2_outputs = [enc2_news_outputs, enc2_sns_outputs, enc2_trend_outputs]
            enc2_states = [enc2_news_states, enc2_sns_states, enc2_trend_states]
            dec_inputs = [dec_news_inputs, dec_sns_inputs, dec_trend_inputs]
            dec_attn_concat = Concatenate(name='decattn_concat')([dec_news_attn, dec_sns_attn, dec_trend_attn])
            
            
        dec_dense1 = TimeDistributed(Dense(DENSE_UNIT, name='dec_dense1'), name='dec_time1')
        dec_dense2 = TimeDistributed(Dense(1, name='dec_dense2'), name='dec_time2')
        leaky_relu = LeakyReLU(alpha=0.1)
        dec_ouputs = dec_dense1(dec_attn_concat)
        dec_ouputs = dec_dense2(dec_ouputs)
        dec_ouputs = leaky_relu(dec_ouputs)
        
        if is_sg:
            model = Model([enc2_inputs, dec_inputs], dec_ouputs)
        else:
            model = Model([enc1_inputs, enc2_inputs, dec_inputs], dec_ouputs)
        
        model.compile(optimizer='adam', loss='mse')
        self.model = model
        
        if mode == 'A':
            dec_news_state_inputs = [
                Input(shape=(dec_lstm_unit,), name='dec_news_h_input'),
                Input(shape=(dec_lstm_unit,), name='dec_news_c_input')
            ]
            dec_sns_state_inputs = [
                Input(shape=(dec_lstm_unit,), name='dec_sns_h_input'),
                Input(shape=(dec_lstm_unit,), name='dec_sns_c_input')
            ]
            dec_trend_state_inputs = [
                Input(shape=(dec_lstm_unit,), name='dec_trend_h_input'),
                Input(shape=(dec_lstm_unit,), name='dec_trend_c_input')
            ]
            
            dec_news_hidden_inputs = Input(shape=(self.look_back, dec_lstm_unit), name='dec_news_hidden_input')
            dec_sns_hidden_inputs = Input(shape=(self.look_back, dec_lstm_unit), name='dec_sns_hidden_input')
            dec_trend_hidden_inputs = Input(shape=(self.look_back, dec_lstm_unit), name='dec_trend_hidden_input')
            
            dec_news_output2, dec_news_state_h, dec_news_state_c = dec_news_lstm1(
                dec_news_inputs, initial_state=dec_news_state_inputs
            )
            dec_sns_output2, dec_sns_state_h, dec_sns_state_c = dec_sns_lstm1(
                dec_sns_inputs, initial_state=dec_sns_state_inputs
            )
            dec_trend_output2, dec_trend_state_h, dec_trend_state_c = dec_trend_lstm1(
                dec_trend_inputs, initial_state=dec_trend_state_inputs
            )
            dec_news_attn2 = Attention(name='dec_news_attn2')([dec_news_output2, dec_news_hidden_inputs])
            dec_sns_attn2 = Attention(name='dec_sns_attn2')([dec_sns_output2, dec_sns_hidden_inputs])
            dec_trend_attn2 = Attention(name='dec_trend_attn2')([dec_trend_output2, dec_trend_hidden_inputs])
            
            dec_attn_concat2 = Concatenate(name='dec_attn_concat2')([
                dec_news_attn2, dec_sns_attn2, dec_trend_attn2
            ])
            dec_outputs2 = dec_dense1(dec_attn_concat2)
            dec_outputs2 = dec_dense2(dec_outputs2)
            dec_outputs2 = leaky_relu(dec_outputs2)
            
            dec_inputs = [dec_news_inputs, dec_sns_inputs, dec_trend_inputs]
            dec_hidden_inputs = [dec_news_hidden_inputs, dec_sns_hidden_inputs, dec_trend_hidden_inputs]
            
            self.decoder_model = Model([dec_news_inputs, dec_sns_inputs, dec_trend_inputs]
                                      + [dec_news_hidden_inputs, dec_sns_hidden_inputs, dec_trend_hidden_inputs]
                                      + [dec_news_state_inputs, dec_sns_state_inputs, dec_trend_state_inputs],
                                      [dec_outputs2] 
                                       + [[dec_news_state_h, dec_news_state_c],
                                          [dec_sns_state_h, dec_sns_state_c],
                                          [dec_trend_state_h, dec_trend_state_c]])
        else:
            dec_state_inputs = [
                Input(shape=(dec_lstm_unit,), name='dec_h_input'),
                Input(shape=(dec_lstm_unit,), name='dec_c_input')
            ]

            dec_hidden_inputs = Input(shape=(self.look_back, dec_lstm_unit), name='dec_hidden_input')

            dec_outputs2, dec_state_h, dec_state_c = dec_lstm1(dec_inputs[0],
                                                                initial_state=dec_state_inputs)
            dec_attn2 = Attention(name='dec_test_attn2')([dec_outputs2, dec_hidden_inputs])
            dec_outputs2 = dec_dense1(dec_attn2)
            dec_outputs2 = dec_dense2(dec_outputs2)
            dec_outputs2 = leaky_relu(dec_outputs2)
            
            self.decoder_model = Model([dec_inputs] + [dec_hidden_inputs] + dec_state_inputs,
                                   [dec_outputs2] + [dec_state_h, dec_state_c])
        
        if is_sg:
          self.encoder_model = Model([enc2_inputs], [enc2_outputs] + enc2_states)
        else:
          self.encoder_model = Model([enc1_inputs, enc2_inputs], [enc2_outputs] + enc2_states)
    
    def fit(self, x, y, val_x, val_y, epochs=10, batch_size=32, callbacks=None, verbose=1):
        history = self.model.fit(x, y, epochs=epochs, batch_size=batch_size,
                                validation_data=(val_x, val_y), callbacks=callbacks,
                                verbose=verbose)
        return history
    
    def load_model(self, model_path=None):
        self.model = load_model(model_path)
        
        if not self.is_sg:
          self.encoder_model.get_layer('enc1_lstm1').set_weights(
              self.model.get_layer('enc1_lstm1').get_weights())
        self.decoder_model.get_layer('dec_time1').set_weights(
            self.model.get_layer('dec_time1').get_weights())
        self.decoder_model.get_layer('dec_time2').set_weights(
            self.model.get_layer('dec_time2').get_weights())
        
        if self.mode == 'N':
            web_types = ['news']
        elif self.mode == 'S':
            web_types = ['sns']
        elif self.mode == 'T':
            web_types = ['trend']
        else:
            web_types = ['news', 'sns', 'trend']
        
        for target_name in web_types:
            self.encoder_model.get_layer('enc2_' + target_name + '_lstm1').set_weights(
                self.model.get_layer('enc2_' + target_name + '_lstm1').get_weights())
            self.decoder_model.get_layer('dec_' + target_name + '_lstm1').set_weights(
                self.model.get_layer('dec_' + target_name + '_lstm1').get_weights())
            
        
    def transform_data(self, truth, news_data, sns_data, trends_data, 
                       news_lag, sns_lag, trends_lag,
                       year_week, look_back, look_ahead):
        
        def _ground_truth(week, year, look_ahead):
            week = int(week)
            max_week = 52
            if week+look_ahead-1 > max_week:
                tmp1 = truth.loc[week:max_week, '20'+year]
                tmp2 = truth.loc[1:look_ahead-max_week+week-1, '20'+str(int(year)+1)]
                ground_truth = pd.concat((tmp1, tmp2))
            else:
                ground_truth = truth.loc[week:week+look_ahead-1, '20'+year]
            return ground_truth.values.reshape(-1, 1)
        
        def _encoder1(year, week, look_back):
            week = int(week)
            if week > look_back:
                tmp1 = truth.loc[week-look_back:week-1,
                                               [str(y) for y in range(2011, 2017)]]
                tmp2 = truth.loc[week-look_back:week-1, '20'+year]
                encoder1_input = pd.concat((tmp1, tmp2), axis=1)
            else:
                max_week = 52
                tmp1 = truth.loc[max_week-look_back+week:max_week,
                                     [str(y) for y in range(2010, 2016)]]
                tmp2 = truth.loc[:week-1,
                                     [str(y) for y in range(2011, 2017)]]
                tmp2.columns = [str(y) for y in range(2010, 2016)]
                tmp2 = pd.concat((tmp1, tmp2))

                tmp3 = truth.loc[max_week-look_back+week:max_week, '20'+str(int(year)-1)]
                tmp4 = truth.loc[:week-1, '20'+year]
                tmp4.columns = [['20'+str(int(year)-1)]]
                tmp4 = pd.concat((tmp3, tmp4))
                encoder1_input = pd.concat((tmp2, tmp4), axis=1)
            return encoder1_input.values
        
        def _encoder2(year_week, look_back):
            if self.mode == 'N':
                web_data = [news_data]
                web_lag = [news_lag]
            elif self.mode == 'S':
                web_data = [sns_data]
                web_lag = [sns_lag]
            elif self.mode == 'T':
                web_data = [trends_data]
                web_lag = [trends_lag]
            else:
                web_data = [news_data, sns_data, trends_data]
                web_lag = [news_lag, sns_lag, trends_lag]
            

            encoder2_input = []
            for data, lag_df in zip(web_data, web_lag):
                tmp = []
                for w in lag_df.columns:
                    lag = int(lag_df[w]['max_lag'])
                    origin_point = data[data['weeks'] == year_week].index[0]
                    start_point = origin_point-lag-look_back
                    end_point = start_point+look_back-1

                    tmp.append(
                            data.loc[start_point:end_point, w].apply(
                            lambda x: np.random.uniform(1e-6, 1e-4) if x == 0 else x).tolist())
                tmp = np.array(tmp).T
                encoder2_input.append(tmp)
            if len(encoder2_input) == 1:
                encoder2_input = tmp
                
            return encoder2_input
        
        def _decoder(year_week, look_ahead):
            year, week = year_week.split('-')
            max_week = 52
            week = int(week)
            if week > 1:
                week = week - 1
            else:
                week = 52
                year = str(int(year) - 1)

            if week+look_ahead-1 > max_week:
                tmp1 = truth.loc[week:max_week, '20'+year]
                tmp2 = truth.loc[1:look_ahead-max_week+week-1, '20'+str(int(year)+1)]
                ground_truth = pd.concat((tmp1, tmp2))
            else:
                ground_truth = truth.loc[week:week+look_ahead-1, '20'+year]

            ground_truth = ground_truth.values.reshape(-1, 1)
            
            if self.mode == 'N':
                web_data = [news_data]
                web_lag = [news_lag]
            elif self.mode == 'S':
                web_data = [sns_data]
                web_lag = [sns_lag]
            elif self.mode == 'T':
                web_data = [trends_data]
                web_lag = [trends_lag]
            else:
                web_data = [news_data, sns_data, trends_data]
                web_lag = [news_lag, sns_lag, trends_lag]
            
            decoder_input = []
            
            for data, lag_df in zip(web_data, web_lag):
                tmp = []
                for i, w in enumerate(list(lag_df.columns)):
                    lag = int(lag_df[w]['max_lag'])
                    origin_point = data[data['weeks'] == year_week].index[0]
                    start_point = origin_point-lag
                    end_point = start_point+look_ahead-1 if lag >= look_ahead else origin_point

                    target_list = data.loc[start_point:end_point, w].tolist()
                    random_pad = np.random.uniform(1e-6, 1e-4, (look_ahead,))
                    for idx in range(len(target_list)):
                        if target_list[idx] == 0:
                            continue
                        random_pad[idx] = target_list[idx]

                    tmp.append(random_pad)
                tmp = np.array(tmp).T
                tmp = np.concatenate([tmp, ground_truth], axis=-1)
                decoder_input.append(tmp)
            if len(decoder_input) == 1:
                decoder_input = tmp
            
            return decoder_input
        
        year, week = year_week.split('-')
        ground_truth = _ground_truth(week, year, look_ahead)
        encoder1_input = _encoder1(year, week, look_back)
        encoder2_input = _encoder2(year_week, look_back)
        decoder_input = _decoder(year_week, look_ahead)      

        return encoder1_input, encoder2_input, decoder_input, ground_truth    
                
        
    def generate_data(self, truth, available_weeks, look_back, look_ahead,
                      news_data=None, sns_data=None, trends_data=None,
                      news_lag=None, sns_lag=None, trends_lag=None,
                      val_split_size=0.8, test_split_size=0.8):
        
        test_size = int(len(available_weeks) * test_split_size)
        val_size = int(test_size * val_split_size)
        
        enc1_inputs = []
        enc2_inputs = []
        dec_inputs = []
        ground_truth = []
        
        enc2_news_inputs = []
        enc2_sns_inputs = []
        enc2_trend_inputs = []
        dec_news_inputs = []
        dec_sns_inputs = []
        dec_trend_inputs = []
        
        for aw in available_weeks:
            e1, e2, d, gt = self.transform_data(truth, 
                                                news_data, sns_data, trends_data,
                                                news_lag, sns_lag, trends_lag,
                                                aw, look_back, look_ahead)
            
            enc1_inputs.append(e1)
            ground_truth.append(gt)
            
            if self.mode == 'A':
                enc2_news_inputs.append(e2[0])
                enc2_sns_inputs.append(e2[1])
                enc2_trend_inputs.append(e2[2])
                dec_news_inputs.append(d[0])
                dec_sns_inputs.append(d[1])
                dec_trend_inputs.append(d[2])
            else:
                enc2_inputs.append(e2)
                dec_inputs.append(d)
            
            
        if self.mode == 'A':
            x = [enc1_inputs,
                 enc2_news_inputs, enc2_sns_inputs, enc2_trend_inputs,
                 dec_news_inputs, dec_sns_inputs, dec_trend_inputs]
        else:
          if self.is_sg:
            x = [np.concatenate([np.array(enc2_inputs),
                                 np.array(enc1_inputs)[:, :, -1:]], axis=-1),
                 dec_inputs]
          else:
            enc1_inputs = np.array(enc1_inputs)[:, :, -1-self.past_year_num:]
            x = [enc1_inputs, enc2_inputs, dec_inputs]
        
        x_total = [np.array(e) for e in x]
        x_train = [np.array(e[:val_size]) for e in x]
        x_val = [np.array(e[val_size:test_size]) for e in x]
        x_test = [np.array(e[test_size:]) for e in x]

        y = np.array(ground_truth)
        y_train = y[:val_size]
        y_val = y[val_size:test_size]
        y_test = y[test_size:]

        return [x_train, x_val, x_test, x_total], [y_train, y_val, y_test], val_size, test_size
            

In [ ]:
class Basic_LSTM():
    def __init__(self, **kwargs):
        LSTM_UNIT = 64
        DENSE_UNIT = 64
        
        self.look_back = kwargs['look_back']
        self.look_ahead = kwargs['look_ahead']
        
        inputs = Input(shape=(self.look_back, 1), name='input1')
        lstm1 = LSTM(LSTM_UNIT, return_state=True, name='lstm1')
        dense1 = Dense(DENSE_UNIT, name='dense1')
        dense2 = Dense(1)
        leaky_relu = LeakyReLU(alpha=0.1)
        
        outputs, _, _ = lstm1(inputs)
        outputs = dense1(outputs)
        outputs = dense2(outputs)
        outputs = leaky_relu(outputs)
        
        model = Model([inputs], outputs)
        model.compile(optimizer='adam', loss='mse')
        self.model = model
        
    def fit(self, x, y, val_x, val_y, epochs=10, batch_size=32, callbacks=None, verbose=1):
        history = self.model.fit(x, y, epochs=epochs, batch_size=batch_size,
                                validation_data=(val_x, val_y), callbacks=callbacks,
                                verbose=verbose)
        return history
    
    def load_model(self, model_path=None):
        self.model = load_model(model_path)
        
    def transform_data(self, truth, year_week, look_back, look_ahead):
        def _ground_truth(week, year, look_ahead):
            week = int(week)
            max_week = 52
            if week+look_ahead-1 > max_week:
                tmp1 = truth.loc[week:max_week, '20'+year]
                tmp2 = truth.loc[1:look_ahead-max_week+week-1, '20'+str(int(year)+1)]
                ground_truth = pd.concat((tmp1, tmp2))
            else:
                ground_truth = truth.loc[week:week+look_ahead-1, '20'+year]
            return ground_truth.values.reshape(-1, 1)
        
        def _inputs(year, week, look_back):
            week = int(week)
            if week > look_back:
                inputs = truth.loc[week-look_back:week-1, '20'+year]
            else:
                max_week = 52
                tmp1 = truth.loc[max_week-look_back+week:max_week, '20'+str(int(year)-1)]
                tmp2 = truth.loc[:week-1, '20'+year]
                tmp2.columns = [['20'+str(int(year)-1)]]
                inputs = pd.concat((tmp1, tmp2))
            return inputs.values.reshape(-1, 1)
        
        year, week = year_week.split('-')
        ground_truth = _ground_truth(week, year, look_ahead)
        inputs = _inputs(year, week, look_back)
        
        return inputs, ground_truth
    
    def generate_data(self, truth, available_weeks, look_back, look_ahead,
                      val_split_size=0.8, test_split_size=0.8):
        
        test_size = int(len(available_weeks) * test_split_size)
        val_size = int(test_size * val_split_size)
        
        ground_truth = []
        inputs = []
        
        for aw in available_weeks:
            i, gt = self.transform_data(truth, aw, look_back, look_ahead)
            inputs.append(i)
            ground_truth.append(gt)
        
        x = [inputs]
        
        x_total = [np.array(e) for e in x]
        x_train = [np.array(e[:val_size]) for e in x]
        x_val = [np.array(e[val_size:test_size]) for e in x]
        x_test = [np.array(e[test_size:]) for e in x]

        y = np.array(ground_truth)
        y_train = y[:val_size]
        y_val = y[val_size:test_size]
        y_test = y[test_size:]

        return [x_train, x_val, x_test, x_total], [y_train, y_val, y_test], val_size, test_size            

In [ ]:
class DEFSI():
    def __init__(self, **kwargs):
        LSTM_UNIT = 64
        DENSE_UNIT = 64
        
        self.look_back = kwargs['look_back']
        self.look_ahead = kwargs['look_ahead']
        
        x1 = Input(shape=(self.look_back, 1), name='x1')
        left_lstm1 = LSTM(LSTM_UNIT, return_sequences=True, return_state=True, name='left_lstm1')
        left_lstm2 = LSTM(LSTM_UNIT, return_state=True, name='left_lstm2')
        left_dense1 = Dense(DENSE_UNIT, name='left_dense1')
        
        left_outputs, _, _ = left_lstm1(x1)
        left_outputs, _, _ = left_lstm2(left_outputs)
        left_outputs = left_dense1(left_outputs)
        
        x2 = Input(shape=(self.look_back, 1), name='x2')
        right_lstm1 = LSTM(LSTM_UNIT, return_state=True, name='right_lsmt1')
        right_dense1 = Dense(DENSE_UNIT, )
        
        right_outputs, _, _ = right_lstm1(x2)
        right_outputs = right_dense1(right_outputs)
        
        merge = Average(name='merge')([left_outputs, right_outputs])
        merge_dense = Dense(1, name='merge_dense')
        outputs = merge_dense(merge)
        
        model = Model([x1, x2], outputs)
        model.compile(optimizer='adam', loss='mse')
        
        self.model = model
    
    def fit(self, x, y, val_x, val_y, epochs=10, batch_size=32, callbacks=None, verbose=1):
        history = self.model.fit(x, y, epochs=epochs, batch_size=batch_size,
                                validation_data=(val_x, val_y), callbacks=callbacks,
                                verbose=verbose)
        return history
    
    def load_model(self, model_path=None):
        self.model = load_model(model_path)
        
    def transform_data(self, truth, year_week, look_back, look_ahead):
        def _ground_truth(week, year, look_ahead):
            week = int(week)
            max_week = 52
            if week+look_ahead-1 > max_week:
                tmp1 = truth.loc[week:max_week, '20'+year]
                tmp2 = truth.loc[1:look_ahead-max_week+week-1, '20'+str(int(year)+1)]
                ground_truth = pd.concat((tmp1, tmp2))
            else:
                ground_truth = truth.loc[week:week+look_ahead-1, '20'+year]
            return ground_truth.values.reshape(-1, 1)
        
        def _inputs(year, week, look_back):
            week = int(week)
            if week > look_back:
                x1 = truth.loc[week-look_back:week-1, '20'+year]
                x2 = truth.loc[week-look_back:week-1, '20'+str(int(year)-1)]
                inputs = [x1.values.reshape(-1, 1), x2.values.reshape(-1, 1)]
            else:
                max_week = 52
                tmp1 = truth.loc[max_week-look_back+week:max_week, '20'+str(int(year)-1)]
                tmp2 = truth.loc[:week-1, '20'+year]
                tmp2.columns = [['20'+str(int(year)-1)]]
                x1 = pd.concat((tmp1, tmp2)).values

                tmp3 = truth.loc[max_week-look_back+week:max_week, '20'+str(int(year)-2)]
                tmp4 = truth.loc[:week-1, '20'+str(int(year)-1)]
                tmp4.columns = [['20'+str(int(year)-2)]]
                x2 = pd.concat((tmp3, tmp4)).values
                inputs = [x1.reshape(-1, 1), x2.reshape(-1, 1)]
            return inputs
        
        year, week = year_week.split('-')
        ground_truth = _ground_truth(week, year, look_ahead)
        inputs = _inputs(year, week, look_back)
        
        return inputs, ground_truth
    
    def generate_data(self, truth, available_weeks, look_back, look_ahead,
                      val_split_size=0.8, test_split_size=0.8):
        
        test_size = int(len(available_weeks) * test_split_size)
        val_size = int(test_size * val_split_size)
        
        ground_truth = []
        x1 = []
        x2 = []
        
        for aw in available_weeks:
            i, gt = self.transform_data(truth, aw, look_back, look_ahead)
            x1.append(i[0])
            x2.append(i[1])
            ground_truth.append(gt)
        
        x = [x1, x2]
        
        x_total = [np.array(e) for e in x]
        x_train = [np.array(e[:val_size]) for e in x]
        x_val = [np.array(e[val_size:test_size]) for e in x]
        x_test = [np.array(e[test_size:]) for e in x]

        y = np.array(ground_truth)
        y_train = y[:val_size]
        y_val = y[val_size:test_size]
        y_test = y[test_size:]

        return [x_train, x_val, x_test, x_total], [y_train, y_val, y_test], val_size, test_size 

In [ ]:
class Seq2Seq():
    def __init__(self, **kwargs):
        ENC_LSTM_UNIT = 64
        DEC_LSTM_UNIT = 128
        DROPOUT = 0.2
        
        self.look_back = kwargs['look_back']
        self.look_ahead = kwargs['look_ahead']
        
        enc_inputs = Input(shape=(self.look_back, 1), name='enc_input')
        enc_lstm1 = Bidirectional(LSTM(ENC_LSTM_UNIT, return_sequences=True, return_state=True,
                                       dropout=DROPOUT, name='enc_lstm1'), name='enc_bilstm1')
        enc_lstm2 = Bidirectional(LSTM(ENC_LSTM_UNIT, return_sequences=True, return_state=True,
                                       dropout=DROPOUT, name='enc_lstm2'), name='enc_bilstm2')
        enc_lstm3 = Bidirectional(LSTM(ENC_LSTM_UNIT, return_sequences=True, return_state=True,
                                       dropout=DROPOUT, name='enc_lstm3'), name='enc_bilstm3')
        enc_outputs, _, _, _, _ = enc_lstm1(enc_inputs)
        enc_outputs, _, _, _, _ = enc_lstm2(enc_outputs)
        enc_outputs, fh, fc, bh, bc = enc_lstm3(enc_outputs)
        
        state_h = Concatenate(name='enc_h_concat')([fh, bh])
        state_c = Concatenate(name='enc_c_concat')([fc, bc])
        
        dec_inputs = Input(shape=(None, 1), name='dec_input')
        dec_lstm1 = LSTM(DEC_LSTM_UNIT, return_sequences=True, return_state=True,
                         dropout=DROPOUT, name='dec_lstm1')
        dec_outputs, _, _ = dec_lstm1(dec_inputs, initial_state=[state_h, state_c])
        
        dec_attn = Attention(name='dec_attn')([dec_outputs, enc_outputs])
        dec_concat = Concatenate(name='dec_concat')([dec_outputs, dec_attn])
        dec_dense1 = TimeDistributed(Dense(1, name='dense1'), name='time1')
        outputs = dec_dense1(dec_concat)
        
        model = Model([enc_inputs, dec_inputs], outputs)
        model.compile(optimizer='adam', loss='mse')
        
        self.model = model
        
        dec_state_inputs = [
            Input(shape=(DEC_LSTM_UNIT,), name='dec_h_input'),
            Input(shape=(DEC_LSTM_UNIT,), name='dec_c_input')
        ]
        dec_hidden_inputs = Input(shape=(self.look_back, DEC_LSTM_UNIT), name='dec_hidden_input')
        
        dec_outputs2, dec_state_h, dec_state_c = dec_lstm1(dec_inputs, 
                                                          initial_state=dec_state_inputs)
        dec_attn2 = Attention(name='dec_test_attn2')([dec_outputs2, dec_hidden_inputs])
        dec_concat2 = Concatenate(name='dec_test_concat2')([dec_outputs2, dec_attn2])
        dec_outputs2 = dec_dense1(dec_concat2)
        
        self.encoder_model = Model(enc_inputs, [enc_outputs, state_h, state_c])
        self.decoder_model = Model([dec_inputs] + [dec_hidden_inputs] + dec_state_inputs,
                                   [dec_outputs2] + [dec_state_h, dec_state_c])
        
    def fit(self, x, y, val_x, val_y, epochs=10, batch_size=32, callbacks=None, verbose=1):
        history = self.model.fit(x, y, epochs=epochs, batch_size=batch_size,
                                validation_data=(val_x, val_y), callbacks=callbacks,
                                verbose=verbose)
        return history
    
    def load_model(self, model_path=None):
        self.model = load_model(model_path)
        
        self.encoder_model.get_layer('enc_bilstm1').set_weights(
            self.model.get_layer('enc_bilstm1').get_weights())
        self.encoder_model.get_layer('enc_bilstm2').set_weights(
            self.model.get_layer('enc_bilstm2').get_weights())
        self.encoder_model.get_layer('enc_bilstm3').set_weights(
            self.model.get_layer('enc_bilstm3').get_weights())
        self.decoder_model.get_layer('dec_lstm1').set_weights(
            self.model.get_layer('dec_lstm1').get_weights())
        self.decoder_model.get_layer('time1').set_weights(
            self.model.get_layer('time1').get_weights())  
        
    def transform_data(self, truth, year_week, look_back, look_ahead):
        def _ground_truth(week, year, look_ahead):
            week = int(week)
            max_week = 52
            if week+look_ahead-1 > max_week:
                tmp1 = truth.loc[week:max_week, '20'+year]
                tmp2 = truth.loc[1:look_ahead-max_week+week-1, '20'+str(int(year)+1)]
                ground_truth = pd.concat((tmp1, tmp2))
            else:
                ground_truth = truth.loc[week:week+look_ahead-1, '20'+year]
            return ground_truth.values.reshape(-1, 1)
        
        def _inputs(year, week, look_back):
            week = int(week)
            if week > look_back:
                inputs = truth.loc[week-look_back:week-1, '20'+year]
            else:
                max_week = 52
                tmp1 = truth.loc[max_week-look_back+week:max_week, '20'+str(int(year)-1)]
                tmp2 = truth.loc[:week-1, '20'+year]
                tmp2.columns = [['20'+str(int(year)-1)]]
                inputs = pd.concat((tmp1, tmp2))
            return inputs.values.reshape(-1, 1)
        
        year, week = year_week.split('-')
        ground_truth = _ground_truth(week, year, look_ahead)
        inputs = _inputs(year, week, look_back)
        inputs = [inputs, np.concatenate([inputs[-1, :].reshape(-1, 1), ground_truth[:-1, :]])]
        
        return inputs, ground_truth        
    
    def generate_data(self, truth, available_weeks, look_back, look_ahead,
                      val_split_size=0.8, test_split_size=0.8):
        
        test_size = int(len(available_weeks) * test_split_size)
        val_size = int(test_size * val_split_size)
        
        ground_truth = []
        enc_inputs = []
        dec_inputs = []
        
        for aw in available_weeks:
            i, gt = self.transform_data(truth, aw, look_back, look_ahead)
            enc_inputs.append(i[0])
            dec_inputs.append(i[1])
            ground_truth.append(gt)
        
        x = [enc_inputs, dec_inputs]
        
        x_total = [np.array(e) for e in x]
        x_train = [np.array(e[:val_size]) for e in x]
        x_val = [np.array(e[val_size:test_size]) for e in x]
        x_test = [np.array(e[test_size:]) for e in x]

        y = np.array(ground_truth)
        y_train = y[:val_size]
        y_val = y[val_size:test_size]
        y_test = y[test_size:]

        return [x_train, x_val, x_test, x_total], [y_train, y_val, y_test], val_size, test_size 

In [ ]:
# Hyper parameter

look_back = 8
look_ahead = 10
past_year_num = 6

max_lag = 14
collect_err = 22

test_split_size = 0.65
val_split_size = 0.8

epochs = 300
batch_size = 32

In [ ]:
perform_by_num_words = {
    'US': {},
    'AU': {},
    'KR': {},
}

In [ ]:
# US_trends MEDIF 적용
epochs = 100
is_sg = False
sg_path = '_2y' #if is_sg else ''

US_perform_by_model = {}
US_prd_by_model = {}

for i in [1]:
#     perform_by_num_words['US'][i] = {}
    mode = 'T'

    mode_name = 'ME-' + mode + '(US)'
    mode_name = mode_name+'(2Y)' #if is_sg else mode_name
    
    US_perform_by_model[mode_name] = {}
    print('mode:', mode_name)
    clear_session()
    
    target_max_lag = US_trends_max_lag.T.sort_values(by='coef', ascending=False).head(i).T
    print(i, target_max_lag.columns)
    trend_word_num = len(target_max_lag.columns)
    trend_word_num = trend_word_num+1 if is_sg else trend_word_num

    me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                          past_year_num=0,
                          trend_word_num=trend_word_num,
                          mode=mode,
                          is_sg=is_sg)

    available_weeks = US_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
    x, y, val_size, test_size = me.generate_data(CDC_norm, available_weeks,
                                                 look_back, look_ahead,
                                                 test_split_size=test_split_size,
                                                 val_split_size=val_split_size,
                                                 trends_data=US_trends_norm,
                                                 trends_lag=target_max_lag)

    x_train, x_val, x_test, x_total = x
    y_train, y_val, y_test = y

    print('val size:', val_size, 'test_size:', test_size)

    cp = ModelCheckpoint('./models/v2/me_{epoch}.h5')

    history = me.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                      callbacks=[cp], verbose=0)

    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()

    min_epoch = 90
    target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                    min(history.history['val_loss'][min_epoch:]))
    best_path = './models/v2/US_me_' + mode + '_lb' + str(look_back) + sg_path + '_best.h5'
    print(target_epoch, best_path)
    shutil.copy('./models/v2/me_' + str(target_epoch) + '.h5', best_path)


    test_me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                              past_year_num=0,
                              trend_word_num=trend_word_num,
                              mode=mode, 
                              is_sg=is_sg)
    test_me.load_model(best_path)

    start_week = int(available_weeks[0].split('-')[1])
    end_week = int(available_weeks[-1].split('-')[1])
    target_truth = (CDC_norm.loc[start_week:52, '2018'].tolist() + CDC_norm.loc[:52, '2019'].tolist()
                    + CDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

    prd_total = []

    for target_idx in range(90-1):
        if is_sg:
          enc_inputs = [x_total[0][target_idx:target_idx+1]]
          dec_input = x_total[1][target_idx, 0, :].reshape(1, 1, x_total[1].shape[2])
        else:
          enc_inputs = [x_total[0][target_idx:target_idx+1],
                        x_total[1][target_idx:target_idx+1]]
          dec_input = x_total[2][target_idx, 0, :].reshape(1, 1, x_total[2].shape[2])
          
        enc_out, enc_h, enc_c = test_me.encoder_model.predict(enc_inputs)

        tmp = []
        for ahead in range(look_ahead):
            prd, dec_h, dec_c = test_me.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
            prd_val = prd[0, 0, 0]
            if prd_val > 0:
                tmp.append([prd_val])
            else:
                tmp.append([np.random.uniform(1e-6, 1e-5)])

            if ahead < look_ahead - 1:
              if is_sg:
                dec_input = x_total[1][target_idx, ahead+1, :].reshape(1, 1, x_total[1].shape[2])
              else:
                dec_input = x_total[2][target_idx, ahead+1, :].reshape(1, 1, x_total[2].shape[2])
              dec_input[0, 0, -1] = prd_val
            enc_h, enc_c = dec_h, dec_c
        prd_total.append(tmp)
    prd_total = np.array(prd_total)
    US_prd_by_model[mode_name] = prd_total

    show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
    prd_train = prd_total[:val_size]
    prd_val = prd_total[val_size:test_size]
    prd_test = prd_total[test_size:]

    inverse_target_truth = CDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
    inverse_prd_total = np.array([CDC_scaler.inverse_transform(_) for _ in prd_total])

    rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                           inverse_prd_total, look_ahead)
    rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                           inverse_prd_total, look_ahead)
    rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                           inverse_prd_total[:val_size], look_ahead)
    rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                     inverse_prd_total[val_size:test_size], look_ahead)
    rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                        inverse_prd_total[test_size:], look_ahead)

    for ahead in range(look_ahead):
        ah = str(ahead+1)
        US_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
        US_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
        US_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

        US_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
        US_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
        US_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

        US_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
        US_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
        US_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

        US_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
        US_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
        US_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
    perform_by_num_words['US'][i]['perform'][mode_name] = US_perform_by_model[mode_name].copy()
    perform_by_num_words['US'][i]['prd'][mode_name] = prd_total

In [ ]:
pd.concat([US_perform, pd.DataFrame(US_perform_by_model).T])[
  ['total_%s_%s' % (ev, i) for i in [1, 2, 3] for ev in ['rmse', 'mape', 'corr']]].round(2)

In [ ]:
## Basic_LSTM

epochs = 30

# US_perform_by_model = {}
mode_name = 'Basic-LSTM(US)'
US_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

basic_lstm = Basic_LSTM(look_back=look_back, look_ahead=look_ahead)
available_weeks = US_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = basic_lstm.generate_data(CDC_norm, available_weeks,
                                                  look_back, 1, 
                                                  test_split_size=test_split_size,
                                                  val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/basic_lstm_{epoch}.h5')

history = basic_lstm.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 10
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))
best_path = './models/v2/US_basic_lstm_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/basic_lstm_' + str(target_epoch) + '.h5', best_path)

test_basic_lstm = Basic_LSTM(look_back=look_back, look_ahead=look_ahead)
test_basic_lstm.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (CDC_norm.loc[start_week:52, '2018'].tolist() + CDC_norm.loc[:52, '2019'].tolist()
                + CDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())


prd_total = []
for target_idx in range(90-1):    
    tmp = []
    for i in range(look_ahead):
        if i == 0:
            left_shift = np.array([x_total[0][target_idx]])
        else:
            left_shift = np.roll(left_shift, -1)
            left_shift[:, -1, :] = tmp[-1]
        prd = test_basic_lstm.model.predict(left_shift)[0, 0]
        prd = np.random.uniform(1e-6, 1e-5) if prd < 0 else prd
        prd = 1.0 - np.random.uniform(1e-6, 1e-5) if prd > 1 else prd
        tmp.append(prd)
#     print(left_shift)
#     print(tmp)
    prd_total.append(tmp)
prd_total = np.array(prd_total)
prd_total = prd_total.reshape((prd_total.shape[0], prd_total.shape[1], 1))
US_prd_by_model[mode_name] = prd_total
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = CDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([CDC_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    US_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    US_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    US_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    US_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    US_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    US_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    US_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    US_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    US_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    US_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    US_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    US_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
## DEFSI

epochs = 300

# US_perform_by_model = {}
mode_name = 'DEFSI(US)'
US_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

defsi = DEFSI(look_back=look_back, look_ahead=1)
available_weeks = US_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = defsi.generate_data(CDC_norm, available_weeks,
                                              look_back, 1, 
                                              test_split_size=test_split_size,
                                              val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/defsi_{epoch}.h5')

history = defsi.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 100
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))
best_path = './models/v2/US_defsi_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/defsi_' + str(target_epoch) + '.h5', best_path)

test_defsi = DEFSI(look_back=look_back, look_ahead=1)
test_defsi.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (CDC_norm.loc[start_week:52, '2018'].tolist() + CDC_norm.loc[:52, '2019'].tolist()
                + CDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

prd_total = []
for target_idx in range(90-1):    
    tmp = []
    for i in range(look_ahead):
        if i == 0:
            left_shift = np.array([x_total[0][target_idx]])
        else:
            left_shift = np.roll(left_shift, -1)
            left_shift[:, -1, :] = tmp[-1]
        prd = test_defsi.model.predict([left_shift, np.array([x_total[1][target_idx+1]])])[0, 0]
        prd = np.random.uniform(1e-6, 1e-5) if prd < 0 else prd
        tmp.append(prd)
#     print(left_shift)
#     print(tmp)
    prd_total.append(tmp)
prd_total = np.array(prd_total)
prd_total = prd_total.reshape((prd_total.shape[0], prd_total.shape[1], 1))
US_prd_by_model[mode_name] = prd_total
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = CDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([CDC_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    US_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    US_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    US_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    US_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    US_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    US_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    US_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    US_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    US_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    US_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    US_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    US_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
## Seq2Seq

# US_perform_by_model = {}
mode_name = 'Seq2Seq(US)'
US_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

seq2seq = Seq2Seq(look_back=look_back, look_ahead=look_ahead)

available_weeks = US_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = seq2seq.generate_data(CDC_norm, available_weeks,
                                                  look_back, look_ahead, 
                                                  test_split_size=test_split_size,
                                                  val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/seq2seq_{epoch}.h5')

history = seq2seq.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 100
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))

best_path = './models/v2/US_seq2seq_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/seq2seq_' + str(target_epoch) + '.h5', best_path)

test_seq2seq = Seq2Seq(look_back=look_back, look_ahead=look_ahead)
test_seq2seq.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (CDC_norm.loc[start_week:52, '2018'].tolist() + CDC_norm.loc[:52, '2019'].tolist()
                + CDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

prd_total = []

for target_idx in range(90-1):
    enc_out, enc_h, enc_c = test_seq2seq.encoder_model.predict(np.array([x_total[0][target_idx]]))

    dec_input = x_total[1][target_idx][0].reshape(-1, 1)

    tmp = []
    for ahead in range(look_ahead):
        prd, dec_h, dec_c = test_seq2seq.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
        prd_val = prd[0, 0]
        if prd[0, 0] > 0:
            tmp.append(prd_val)
        else:
            tmp.append([np.random.uniform(1e-6, 1e-5)])

        dec_input = prd[0, 0]
        enc_h, enc_c = dec_h, dec_c
    prd_total.append(tmp)
prd_total = np.array(prd_total)
US_prd_by_model[mode_name] = prd_total
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = CDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([CDC_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    US_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    US_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    US_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    US_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    US_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    US_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    US_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    US_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    US_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    US_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    US_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    US_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
US_perform = pd.concat([US_perform, pd.DataFrame(US_perform_by_model).T])

In [ ]:
# US_perform = pd.DataFrame(US_perform_by_model).T
US_perform.to_csv('./data/v2/US_perform6.csv', encoding='utf8')
US_perform

In [ ]:
with open('./data/v2/US_predict4.pkl', 'wb') as f:
    pickle.dump(US_prd_by_model, f)

In [ ]:
# AU_trends MEDIF 적용
epochs = 100
is_sg = False
# sg_path = '_sg' if is_sg else ''
sg_path = '_2y'

AU_perform_by_model = {}
AU_prd_by_model = {}

for i in [1]:
#     perform_by_num_words['AU'][i] = {}
    mode = 'T'

    mode_name = 'ME-' + mode + '(AU)'
    mode_name = mode_name+'(2Y)' # if is_sg else mode_name
    
    AU_perform_by_model[mode_name] = {}
    print('mode:', mode_name)
    clear_session()
    
    target_max_lag = AU_trends_max_lag.T.sort_values(by='coef', ascending=False).head(i).T
    trend_word_num = len(target_max_lag.columns)
    trend_word_num = trend_word_num+1 if is_sg else trend_word_num

    me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                          past_year_num=5,
                          trend_word_num=trend_word_num,
                          mode=mode,
                          is_sg=is_sg)

    available_weeks = AU_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
    x, y, val_size, test_size = me.generate_data(ASPREN_norm, available_weeks,
                                                 look_back, look_ahead,
                                                 test_split_size=test_split_size,
                                                 val_split_size=val_split_size,
                                                 trends_data=AU_trends_norm,
                                                 trends_lag=target_max_lag)

    x_train, x_val, x_test, x_total = x
    y_train, y_val, y_test = y

    print('val size:', val_size, 'test_size:', test_size)

    cp = ModelCheckpoint('./models/v2/me_{epoch}.h5')

    history = me.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                      callbacks=[cp], verbose=0)

    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()

    min_epoch = 45
    target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                    min(history.history['val_loss'][min_epoch:]))
    best_path = './models/v2/AU_me_' + mode + '_lb' + str(look_back) + sg_path + '_best.h5'
    print(target_epoch, best_path)
    shutil.copy('./models/v2/me_' + str(target_epoch) + '.h5', best_path)


    test_me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                              past_year_num=5,
                              trend_word_num=trend_word_num,
                              mode=mode,
                              is_sg=is_sg)
    test_me.load_model(best_path)

    start_week = int(available_weeks[0].split('-')[1])
    end_week = int(available_weeks[-1].split('-')[1])
    target_truth = (ASPREN_norm.loc[start_week:52, '2018'].tolist() + ASPREN_norm.loc[:52, '2019'].tolist()
                    + ASPREN_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

    prd_total = []

    for target_idx in range(90-1):
        if is_sg:
            enc_inputs = [x_total[0][target_idx:target_idx+1]]
            dec_input = x_total[1][target_idx, 0, :].reshape(1, 1, x_total[1].shape[2])
        else:
            enc_inputs = [x_total[0][target_idx:target_idx+1],
                          x_total[1][target_idx:target_idx+1]]
            dec_input = x_total[2][target_idx, 0, :].reshape(1, 1, x_total[2].shape[2])

        enc_out, enc_h, enc_c = test_me.encoder_model.predict(enc_inputs)

        tmp = []
        for ahead in range(look_ahead):
            prd, dec_h, dec_c = test_me.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
            prd_val = prd[0, 0, 0]
            if prd_val > 0:
                tmp.append([prd_val])
            else:
                tmp.append([np.random.uniform(1e-6, 1e-5)])

            if ahead < look_ahead - 1:
              if is_sg:
                dec_input = x_total[1][target_idx, ahead+1, :].reshape(1, 1, x_total[1].shape[2])
              else:
                dec_input = x_total[2][target_idx, ahead+1, :].reshape(1, 1, x_total[2].shape[2])
              dec_input[0, 0, -1] = prd_val
            enc_h, enc_c = dec_h, dec_c
        prd_total.append(tmp)
    prd_total = np.array(prd_total)
    AU_prd_by_model[mode_name] = prd_total

    show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
    prd_train = prd_total[:val_size]
    prd_val = prd_total[val_size:test_size]
    prd_test = prd_total[test_size:]

    inverse_target_truth = ASPREN_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
    inverse_prd_total = np.array([ASPREN_scaler.inverse_transform(_) for _ in prd_total])

    rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                           inverse_prd_total, look_ahead)
    rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                           inverse_prd_total, look_ahead)
    rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                           inverse_prd_total[:val_size], look_ahead)
    rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                     inverse_prd_total[val_size:test_size], look_ahead)
    rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                        inverse_prd_total[test_size:], look_ahead)

    for ahead in range(look_ahead):
        ah = str(ahead+1)
        AU_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
        AU_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
        AU_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

        AU_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
        AU_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
        AU_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

        AU_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
        AU_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
        AU_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

        AU_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
        AU_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
        AU_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
    perform_by_num_words['AU'][i]['perform'][mode_name] = AU_perform_by_model[mode_name].copy()
    perform_by_num_words['AU'][i]['prd'][mode_name] = prd_total

In [ ]:
pd.concat([AU_perform, pd.DataFrame(AU_perform_by_model).T])[['total_%s_%s' % (ev, i) for i in [1,2,3] for ev in ['rmse', 'mape', 'corr']]].round(2)

In [ ]:
## Basic_LSTM

epochs = 30

# AU_perform_by_model = {}
mode_name = 'Basic-LSTM(AU)'
AU_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

basic_lstm = Basic_LSTM(look_back=look_back, look_ahead=look_ahead)
available_weeks = AU_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = basic_lstm.generate_data(ASPREN_norm, available_weeks,
                                                  look_back, 1, 
                                                  test_split_size=test_split_size,
                                                  val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/basic_lstm_{epoch}.h5')

history = basic_lstm.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 10
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))
best_path = './models/v2/AU_basic_lstm_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/basic_lstm_' + str(target_epoch) + '.h5', best_path)

test_basic_lstm = Basic_LSTM(look_back=look_back, look_ahead=look_ahead)
test_basic_lstm.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (ASPREN_norm.loc[start_week:52, '2018'].tolist() + ASPREN_norm.loc[:52, '2019'].tolist()
                + ASPREN_norm.loc[:end_week+look_ahead-1, '2020'].tolist())


prd_total = []
for target_idx in range(90-1):    
    tmp = []
    for i in range(look_ahead):
        if i == 0:
            left_shift = np.array([x_total[0][target_idx]])
        else:
            left_shift = np.roll(left_shift, -1)
            left_shift[:, -1, :] = tmp[-1]
        prd = test_basic_lstm.model.predict(left_shift)[0, 0]
        prd = np.random.uniform(1e-6, 1e-5) if prd < 0 else prd
        prd = 1.0 - np.random.uniform(1e-6, 1e-5) if prd > 1 else prd
        tmp.append(prd)
#     print(left_shift)
#     print(tmp)
    prd_total.append(tmp)
prd_total = np.array(prd_total)
prd_total = prd_total.reshape((prd_total.shape[0], prd_total.shape[1], 1))
AU_prd_by_model[mode_name] = prd_total
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = ASPREN_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([ASPREN_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    AU_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    AU_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    AU_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    AU_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    AU_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    AU_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    AU_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    AU_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    AU_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    AU_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    AU_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    AU_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
## DEFSI

epochs = 300

# AU_perform_by_model = {}
mode_name = 'DEFSI(AU)'
AU_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

defsi = DEFSI(look_back=look_back, look_ahead=1)
available_weeks = AU_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = defsi.generate_data(ASPREN_norm, available_weeks,
                                              look_back, 1, 
                                              test_split_size=test_split_size,
                                              val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/defsi_{epoch}.h5')

history = defsi.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 100
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))
best_path = './models/v2/AU_defsi_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/defsi_' + str(target_epoch) + '.h5', best_path)

test_defsi = DEFSI(look_back=look_back, look_ahead=1)
test_defsi.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (ASPREN_norm.loc[start_week:52, '2018'].tolist() + ASPREN_norm.loc[:52, '2019'].tolist()
                + ASPREN_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

prd_total = []
for target_idx in range(90-1):    
    tmp = []
    for i in range(look_ahead):
        if i == 0:
            left_shift = np.array([x_total[0][target_idx]])
        else:
            left_shift = np.roll(left_shift, -1)
            left_shift[:, -1, :] = tmp[-1]
        prd = test_defsi.model.predict([left_shift, np.array([x_total[1][target_idx+1]])])[0, 0]
        prd = np.random.uniform(1e-6, 1e-5) if prd < 0 else prd
        prd = 1 - np.random.uniform(1e-6, 1e-5) if prd > 1 else prd
        tmp.append(prd)
#     print(left_shift)
#     print(tmp)
    prd_total.append(tmp)
prd_total = np.array(prd_total)
prd_total = prd_total.reshape((prd_total.shape[0], prd_total.shape[1], 1))
AU_prd_by_model[mode_name] = prd_total
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = ASPREN_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([ASPREN_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    AU_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    AU_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    AU_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    AU_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    AU_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    AU_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    AU_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    AU_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    AU_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    AU_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    AU_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    AU_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
## Seq2Seq

# AU_perform_by_model = {}
mode_name = 'Seq2Seq(AU)'
AU_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

seq2seq = Seq2Seq(look_back=look_back, look_ahead=look_ahead)

available_weeks = AU_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = seq2seq.generate_data(ASPREN_norm, available_weeks,
                                                  look_back, look_ahead, 
                                                  test_split_size=test_split_size,
                                                  val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/seq2seq_{epoch}.h5')

history = seq2seq.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 100
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))

best_path = './models/v2/AU_seq2seq_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/seq2seq_' + str(target_epoch) + '.h5', best_path)

test_seq2seq = Seq2Seq(look_back=look_back, look_ahead=look_ahead)
test_seq2seq.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (ASPREN_norm.loc[start_week:52, '2018'].tolist() + ASPREN_norm.loc[:52, '2019'].tolist()
                + ASPREN_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

prd_total = []

for target_idx in range(90-1):
    enc_out, enc_h, enc_c = test_seq2seq.encoder_model.predict(np.array([x_total[0][target_idx]]))

    dec_input = x_total[1][target_idx][0].reshape(-1, 1)

    tmp = []
    for ahead in range(look_ahead):
        prd, dec_h, dec_c = test_seq2seq.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
        prd_val = prd[0, 0]
        if prd[0, 0] > 0:
            tmp.append(prd_val)
        else:
            tmp.append([np.random.uniform(1e-6, 1e-5)])

        dec_input = prd[0, 0]
        enc_h, enc_c = dec_h, dec_c
    prd_total.append(tmp)
prd_total = np.array(prd_total)
AU_prd_by_model[mode_name] = prd_total
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = ASPREN_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([ASPREN_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    AU_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    AU_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    AU_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    AU_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    AU_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    AU_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    AU_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    AU_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    AU_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    AU_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    AU_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    AU_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
AU_perform = pd.concat([AU_perform, pd.DataFrame(AU_perform_by_model).T])

In [ ]:
# AU_perform = pd.DataFrame(AU_perform_by_model).T
AU_perform.to_csv('./data/v2/AU_perform6.csv', encoding='utf8')
AU_perform

In [ ]:
with open('./data/v2/AU_predict4.pkl', 'wb') as f:
    pickle.dump(AU_prd_by_model, f)

In [ ]:
# KR_N,S,T MEDIF , look_back param check

KR_perform_by_lookback = {}
KR_perform_by_model = {}
KR_prd_by_model = {}

mode = 'T'

for look_back in range(3, 11):
    print('look back:', look_back)
    KR_perform_by_lookback[look_back] = {}
    for mode in ['N', 'S', 'T', 'A']:
        mode_name = 'ME-' + mode
        KR_perform_by_model[mode_name] = {}
        print('mode:', mode_name)

        clear_session()

        news_word_num = len(KR_news_max_lag.columns)
        sns_word_num = len(KR_sns_max_lag.columns)
        trend_word_num = len(KR_trends_max_lag.columns)

        me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                              past_year_num=6,
                              news_word_num=news_word_num,
                              sns_word_num=sns_word_num,
                              trend_word_num=trend_word_num,
                              mode=mode)

        available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
        x, y, val_size, test_size = me.generate_data(KCDC_norm, available_weeks,
                                                     look_back, look_ahead,
                                                     test_split_size=test_split_size,
                                                     val_split_size=val_split_size,
                                                     news_data=KR_news_norm,
                                                     news_lag=KR_news_max_lag,
                                                     sns_data=KR_sns_norm,
                                                     sns_lag=KR_sns_max_lag,
                                                     trends_data=KR_trends_norm,
                                                     trends_lag=KR_trends_max_lag)

        x_train, x_val, x_test, x_total = x
        y_train, y_val, y_test = y

        print('val size:', val_size, 'test_size:', test_size)

        cp = ModelCheckpoint('./models/v2/me_{epoch}.h5')

        history = me.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                          callbacks=[cp], verbose=0)

        plt.plot(history.history['loss'], label='train_loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.legend()
        plt.show()

        min_epoch = 100
        target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                        min(history.history['val_loss'][min_epoch:]))
        best_path = './models/v2/KR_me_' + mode + '_lb' + str(look_back) + '_best.h5'
        print(target_epoch, best_path)
        shutil.copy('./models/v2/me_' + str(target_epoch) + '.h5', best_path)


        test_me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                                  past_year_num=past_year_num,
                                  news_word_num=news_word_num,
                                  sns_word_num=sns_word_num,
                                  trend_word_num=trend_word_num,
                                  mode=mode)
        test_me.load_model(best_path)

        start_week = int(available_weeks[0].split('-')[1])
        end_week = int(available_weeks[-1].split('-')[1])
        target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                        + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

        prd_total = []

        for target_idx in range(90-1-look_back+8):
            if mode == 'A':
                enc_outs, news_states, sns_states, trend_states = me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                                            [x_total[1][target_idx:target_idx+1],
                                                                                             x_total[2][target_idx:target_idx+1],
                                                                                             x_total[3][target_idx:target_idx+1]]])

                dec_input = [x_total[4][target_idx, 0, :].reshape(1, 1, x_total[4].shape[2]),
                             x_total[5][target_idx, 0, :].reshape(1, 1, x_total[5].shape[2]),
                             x_total[6][target_idx, 0, :].reshape(1, 1, x_total[6].shape[2])]
                tmp = []
                for ahead in range(look_ahead):
                    prd, news_states, sns_states, trend_states = me.decoder_model.predict(dec_input + enc_outs 
                                                                      + [news_states, sns_states, trend_states])
                    prd_val = prd[0, 0, 0]
                    if prd_val > 0:
                        tmp.append([prd_val])
                    else:
                        tmp.append([np.random.uniform(1e-6, 1e-5)])

                    if ahead < look_ahead - 1:
                        dec_input = [x_total[4][target_idx, ahead+1, :].reshape(1, 1, x_total[4].shape[2]),
                             x_total[5][target_idx, ahead+1, :].reshape(1, 1, x_total[5].shape[2]),
                             x_total[6][target_idx, ahead+1, :].reshape(1, 1, x_total[6].shape[2])]
                        dec_input[0][0, 0, -1] = prd_val
                        dec_input[1][0, 0, -1] = prd_val
                        dec_input[2][0, 0, -1] = prd_val
            else:
                enc_out, enc_h, enc_c = test_me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                      x_total[1][target_idx:target_idx+1]])

                dec_input = x_total[2][target_idx, 0, :].reshape(1, 1, x_total[2].shape[2])
                tmp = []
                for ahead in range(look_ahead):
                    prd, dec_h, dec_c = test_me.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
                    prd_val = prd[0, 0, 0]
                    if prd_val > 0:
                        tmp.append([prd_val])
                    else:
                        tmp.append([np.random.uniform(1e-6, 1e-5)])

                    if ahead < look_ahead - 1:
                        dec_input = x_total[2][target_idx, ahead+1, :].reshape(1, 1, x_total[2].shape[2])
                        dec_input[0, 0, -1] = prd_val
                    enc_h, enc_c = dec_h, dec_c
            prd_total.append(tmp)
        prd_total = np.array(prd_total)
        KR_prd_by_model[mode_name] = prd_total

        show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
        prd_train = prd_total[:val_size]
        prd_val = prd_total[val_size:test_size]
        prd_test = prd_total[test_size:]

        rmse_total, mape_total, corr_total = Evaluate.evaluate(target_truth, 
                                                               prd_total, look_ahead)
        rmse_train, mape_train, corr_train = Evaluate.evaluate(target_truth[:val_size+look_ahead], 
                                                               prd_train, look_ahead)
        rmse_val, mape_val, corr_val = Evaluate.evaluate(target_truth[val_size:test_size+look_ahead],
                                                         prd_val, look_ahead)
        rmse_test, mape_test, corr_test = Evaluate.evaluate(target_truth[test_size:],
                                                            prd_test, look_ahead)

        for ahead in range(look_ahead):
            ah = str(ahead+1)
            KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
            KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
            KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

            KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
            KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
            KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

            KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
            KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
            KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

            KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
            KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
            KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
#     print(KR_perform_by_model)
    KR_perform_by_lookback[look_back]['perform'] = KR_perform_by_model.copy()
    KR_perform_by_lookback[look_back]['predict'] = KR_prd_by_model.copy()

In [ ]:
with open('./data/v2/KR_perform_by_lookback.pkl', 'wb') as f:
    pickle.dump(KR_perform_by_lookback, f)

In [ ]:
KR_perform_by_model = {}

for look_back in range(3, 11):
    for mode in ['N', 'S', 'T']:
        mode_name = 'ME-' + mode
        KR_perform_by_model[mode_name] = {}
        
        available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
        start_week = int(available_weeks[0].split('-')[1])
        end_week = int(available_weeks[-1].split('-')[1])
        
        target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                        + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

        rmse_total, mape_total, corr_total = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1)), 
                                                               np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_lookback[look_back]['predict'][mode_name]]),
                                                               look_ahead)
        rmse_train, mape_train, corr_train = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[:val_size+look_ahead], 
                                                               np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_lookback[look_back]['predict'][mode_name]])[:val_size],
                                                               look_ahead)
        rmse_val, mape_val, corr_val = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[val_size:test_size+look_ahead],
                                                         np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_lookback[look_back]['predict'][mode_name]])[val_size:test_size],
                                                         look_ahead)
        rmse_test, mape_test, corr_test = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[test_size:],
                                                            np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_lookback[look_back]['predict'][mode_name]])[test_size:],
                                                            look_ahead)

        for ahead in range(look_ahead):
            ah = str(ahead+1)
            KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
            KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
            KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

            KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
            KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
            KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

            KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
            KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
            KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

            KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
            KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
            KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
    KR_perform_by_lookback[look_back]['perform'] = KR_perform_by_model.copy()

In [ ]:
lb_compare = {}

for ev in ['rmse', 'mape', 'corr']:
    lb_compare[ev] = []
    for lb in range(3, 11):
        tmp2 = pd.DataFrame(KR_perform_by_lookback[lb]['perform']).T.loc[['ME-N', 'ME-S', 'ME-T']]
        lb_compare[ev].append(tmp2[[c for c in tmp2.columns if 'total' in c and ev in c]].mean(axis=1).mean())

lb_compare = pd.DataFrame(lb_compare, index=[_ for _ in range(3, 11)])
lb_compare['rmse(improve)'] = lb_compare['rmse'].apply(lambda x: (lb_compare['rmse'].max() - x) / lb_compare['rmse'].max())
lb_compare['mape(improve)'] = lb_compare['mape'].apply(lambda x: (lb_compare['mape'].max() - x) / lb_compare['mape'].max())
lb_compare['corr(improve)'] = lb_compare['corr'].apply(lambda x: (x - lb_compare['corr'].min()) / lb_compare['corr'].min())
lb_compare['improve(mean)'] = lb_compare[['rmse(improve)', 'mape(improve)', 'corr(improve)']].mean(axis=1)
lb_compare.to_csv('./data/v2/KR_perform_by_lookback.csv', encoding='utf8')
lb_compare.round(3)

In [ ]:
# KR_N,S,T MEDIF , layer_size param check

KR_perform_by_layersize = {}
KR_perform_by_model = {}
KR_prd_by_model = {}

look_back = 7

mode = 'T'

for layer_size in [32, 64, 128, 256, 512]:
    print('layer_size:', layer_size)
    KR_perform_by_layersize[layer_size] = {}
    for mode in ['N', 'S', 'T', 'A']:
        mode_name = 'ME-' + mode
        KR_perform_by_model[mode_name] = {}
        print('mode:', mode_name)

        clear_session()

        news_word_num = len(KR_news_max_lag.columns)
        sns_word_num = len(KR_sns_max_lag.columns)
        trend_word_num = len(KR_trends_max_lag.columns)

        me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                              past_year_num=6,
                              news_word_num=news_word_num,
                              sns_word_num=sns_word_num,
                              trend_word_num=trend_word_num,
                              layer_size=layer_size,
                              mode=mode)

        available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
        x, y, val_size, test_size = me.generate_data(KCDC_norm, available_weeks,
                                                     look_back, look_ahead,
                                                     test_split_size=test_split_size,
                                                     val_split_size=val_split_size,
                                                     news_data=KR_news_norm,
                                                     news_lag=KR_news_max_lag,
                                                     sns_data=KR_sns_norm,
                                                     sns_lag=KR_sns_max_lag,
                                                     trends_data=KR_trends_norm,
                                                     trends_lag=KR_trends_max_lag)

        x_train, x_val, x_test, x_total = x
        y_train, y_val, y_test = y

        print('val size:', val_size, 'test_size:', test_size)

        cp = ModelCheckpoint('./models/v2/me_{epoch}.h5')

        history = me.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                          callbacks=[cp], verbose=0)

        plt.plot(history.history['loss'], label='train_loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.legend()
        plt.show()

        min_epoch = 100
        target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                        min(history.history['val_loss'][min_epoch:]))
        best_path = './models/v2/KR_me_' + mode + '_lb' + str(look_back) + '_best.h5'
        print(target_epoch, best_path)
        shutil.copy('./models/v2/me_' + str(target_epoch) + '.h5', best_path)


        test_me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                                  past_year_num=past_year_num,
                                  news_word_num=news_word_num,
                                  sns_word_num=sns_word_num,
                                  trend_word_num=trend_word_num,
                                  layer_size=layer_size,
                                  mode=mode)
        test_me.load_model(best_path)

        start_week = int(available_weeks[0].split('-')[1])
        end_week = int(available_weeks[-1].split('-')[1])
        target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                        + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

        prd_total = []

        for target_idx in range(90-1-look_back+8):
            if mode == 'A':
                enc_outs, news_states, sns_states, trend_states = me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                                            [x_total[1][target_idx:target_idx+1],
                                                                                             x_total[2][target_idx:target_idx+1],
                                                                                             x_total[3][target_idx:target_idx+1]]])

                dec_input = [x_total[4][target_idx, 0, :].reshape(1, 1, x_total[4].shape[2]),
                             x_total[5][target_idx, 0, :].reshape(1, 1, x_total[5].shape[2]),
                             x_total[6][target_idx, 0, :].reshape(1, 1, x_total[6].shape[2])]
                tmp = []
                for ahead in range(look_ahead):
                    prd, news_states, sns_states, trend_states = me.decoder_model.predict(dec_input + enc_outs 
                                                                      + [news_states, sns_states, trend_states])
                    prd_val = prd[0, 0, 0]
                    if prd_val > 0:
                        tmp.append([prd_val])
                    else:
                        tmp.append([np.random.uniform(1e-6, 1e-5)])

                    if ahead < look_ahead - 1:
                        dec_input = [x_total[4][target_idx, ahead+1, :].reshape(1, 1, x_total[4].shape[2]),
                             x_total[5][target_idx, ahead+1, :].reshape(1, 1, x_total[5].shape[2]),
                             x_total[6][target_idx, ahead+1, :].reshape(1, 1, x_total[6].shape[2])]
                        dec_input[0][0, 0, -1] = prd_val
                        dec_input[1][0, 0, -1] = prd_val
                        dec_input[2][0, 0, -1] = prd_val
            else:
                enc_out, enc_h, enc_c = test_me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                      x_total[1][target_idx:target_idx+1]])

                dec_input = x_total[2][target_idx, 0, :].reshape(1, 1, x_total[2].shape[2])
                tmp = []
                for ahead in range(look_ahead):
                    prd, dec_h, dec_c = test_me.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
                    prd_val = prd[0, 0, 0]
                    if prd_val > 0:
                        tmp.append([prd_val])
                    else:
                        tmp.append([np.random.uniform(1e-6, 1e-5)])

                    if ahead < look_ahead - 1:
                        dec_input = x_total[2][target_idx, ahead+1, :].reshape(1, 1, x_total[2].shape[2])
                        dec_input[0, 0, -1] = prd_val
                    enc_h, enc_c = dec_h, dec_c
            prd_total.append(tmp)
        prd_total = np.array(prd_total)
        KR_prd_by_model[mode_name] = prd_total

        show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
        prd_train = prd_total[:val_size]
        prd_val = prd_total[val_size:test_size]
        prd_test = prd_total[test_size:]

        rmse_total, mape_total, corr_total = Evaluate.evaluate(target_truth, 
                                                               prd_total, look_ahead)
        rmse_train, mape_train, corr_train = Evaluate.evaluate(target_truth[:val_size+look_ahead], 
                                                               prd_train, look_ahead)
        rmse_val, mape_val, corr_val = Evaluate.evaluate(target_truth[val_size:test_size+look_ahead],
                                                         prd_val, look_ahead)
        rmse_test, mape_test, corr_test = Evaluate.evaluate(target_truth[test_size:],
                                                            prd_test, look_ahead)

        for ahead in range(look_ahead):
            ah = str(ahead+1)
            KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
            KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
            KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

            KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
            KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
            KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

            KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
            KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
            KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

            KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
            KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
            KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
#     print(KR_perform_by_model)
    KR_perform_by_layersize[layer_size]['perform'] = KR_perform_by_model.copy()
    KR_perform_by_layersize[layer_size]['predict'] = KR_prd_by_model.copy()

In [ ]:
with open('./data/v2/KR_perform_by_layersize.pkl', 'wb') as f:
    pickle.dump(KR_perform_by_layersize, f)

In [ ]:
KR_perform_by_model = {}

look_back = 7

for layer_size in [32, 64, 128, 256, 512]:
    for mode in ['N', 'S', 'T']:
        mode_name = 'ME-' + mode
        KR_perform_by_model[mode_name] = {}
        
        available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
        start_week = int(available_weeks[0].split('-')[1])
        end_week = int(available_weeks[-1].split('-')[1])
        
        target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                        + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

        rmse_total, mape_total, corr_total = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1)), 
                                                               np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_layersize[layer_size]['predict'][mode_name]]),
                                                               look_ahead)
        rmse_train, mape_train, corr_train = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[:val_size+look_ahead], 
                                                               np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_layersize[layer_size]['predict'][mode_name]])[:val_size],
                                                               look_ahead)
        rmse_val, mape_val, corr_val = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[val_size:test_size+look_ahead],
                                                         np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_layersize[layer_size]['predict'][mode_name]])[val_size:test_size],
                                                         look_ahead)
        rmse_test, mape_test, corr_test = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[test_size:],
                                                            np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_layersize[layer_size]['predict'][mode_name]])[test_size:],
                                                            look_ahead)

        for ahead in range(look_ahead):
            ah = str(ahead+1)
            KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
            KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
            KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

            KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
            KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
            KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

            KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
            KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
            KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

            KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
            KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
            KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
    KR_perform_by_layersize[layer_size]['perform'] = KR_perform_by_model.copy()

In [ ]:
ls_compare = {}

for ev in ['rmse', 'mape', 'corr']:
    ls_compare[ev] = []
    for ls in [32, 64, 128, 256, 512]:
        tmp2 = pd.DataFrame(KR_perform_by_layersize[ls]['perform']).T.loc[['ME-N', 'ME-S', 'ME-T']]
        ls_compare[ev].append(tmp2[[c for c in tmp2.columns if 'total' in c and ev in c]].mean(axis=1).mean())

ls_compare = pd.DataFrame(ls_compare, index=[32, 64, 128, 256, 512])
ls_compare['rmse(improve)'] = ls_compare['rmse'].apply(lambda x: (ls_compare['rmse'].max() - x) / ls_compare['rmse'].max())
ls_compare['mape(improve)'] = ls_compare['mape'].apply(lambda x: (ls_compare['mape'].max() - x) / ls_compare['mape'].max())
ls_compare['corr(improve)'] = ls_compare['corr'].apply(lambda x: (x - ls_compare['corr'].min()) / ls_compare['corr'].min())
ls_compare['improve(mean)'] = ls_compare[['rmse(improve)', 'mape(improve)', 'corr(improve)']].mean(axis=1)
ls_compare.to_csv('./data/v2/KR_perform_by_layersize.csv', encoding='utf8')
ls_compare.round(3)

In [ ]:
# KR_N,S,T MEDIF , max_lag coef threshold param check

KR_perform_by_coef = {}
KR_perform_by_model = {}
KR_prd_by_model = {}

look_back = 7

mode = 'T'

for threshold in [0.0, 0.1, 0.2, 0.3, 0.4]:
    
    KR_news_tmp_max_lag = {}
    lag_max = KR_news_lag.max()
    lag_max = lag_max[lag_max > threshold]
    for word in lag_max.index:
        lag = KR_news_lag[word].argmax()
        val = KR_news_lag.loc[lag, word]
        if lag >= 0:
            KR_news_tmp_max_lag[word] = (lag, val)
    KR_news_tmp_max_lag = pd.DataFrame(KR_news_tmp_max_lag, index=['max_lag', 'coef'])
    
    KR_sns_tmp_max_lag = {}
    lag_max = KR_sns_lag.max()
    lag_max = lag_max[lag_max > threshold]
    for word in lag_max.index:
        lag = KR_sns_lag[word].argmax()
        val = KR_sns_lag.loc[lag, word]
        if lag >= 0:
            KR_sns_tmp_max_lag[word] = (lag, val)
    KR_sns_tmp_max_lag = pd.DataFrame(KR_sns_tmp_max_lag, index=['max_lag', 'coef'])
    
    KR_trends_tmp_max_lag = {}
    lag_max = KR_trends_lag.max()
    lag_max = lag_max[lag_max > threshold]
    for word in lag_max.index:
        lag = KR_trends_lag[word].argmax()
        val = KR_trends_lag.loc[lag, word]
        if lag >= 0:
            KR_trends_tmp_max_lag[word] = (lag, val)
    KR_trends_tmp_max_lag = pd.DataFrame(KR_trends_tmp_max_lag, index=['max_lag', 'coef'])
    
    print('threshold:', threshold)
    KR_perform_by_coef[threshold] = {}
    for mode in ['N', 'S', 'T', 'A']:
        mode_name = 'ME-' + mode
        KR_perform_by_model[mode_name] = {}
        print('mode:', mode_name)

        clear_session()

        news_word_num = len(KR_news_tmp_max_lag.columns)
        sns_word_num = len(KR_sns_tmp_max_lag.columns)
        trend_word_num = len(KR_trends_tmp_max_lag.columns)

        me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                              past_year_num=6,
                              news_word_num=news_word_num,
                              sns_word_num=sns_word_num,
                              trend_word_num=trend_word_num,
                              mode=mode)

        available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
        x, y, val_size, test_size = me.generate_data(KCDC_norm, available_weeks,
                                                     look_back, look_ahead,
                                                     test_split_size=test_split_size,
                                                     val_split_size=val_split_size,
                                                     news_data=KR_news_norm,
                                                     news_lag=KR_news_tmp_max_lag,
                                                     sns_data=KR_sns_norm,
                                                     sns_lag=KR_sns_tmp_max_lag,
                                                     trends_data=KR_trends_norm,
                                                     trends_lag=KR_trends_tmp_max_lag)

        x_train, x_val, x_test, x_total = x
        y_train, y_val, y_test = y

        print('val size:', val_size, 'test_size:', test_size)

        cp = ModelCheckpoint('./models/v2/me_{epoch}.h5')

        history = me.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                          callbacks=[cp], verbose=0)

        plt.plot(history.history['loss'], label='train_loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.legend()
        plt.show()

        min_epoch = 100
        target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                        min(history.history['val_loss'][min_epoch:]))
        best_path = './models/v2/KR_me_' + mode + '_lb' + str(look_back) + '_best.h5'
        print(target_epoch, best_path)
        shutil.copy('./models/v2/me_' + str(target_epoch) + '.h5', best_path)


        test_me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                                  past_year_num=past_year_num,
                                  news_word_num=news_word_num,
                                  sns_word_num=sns_word_num,
                                  trend_word_num=trend_word_num,
                                  mode=mode)
        test_me.load_model(best_path)

        start_week = int(available_weeks[0].split('-')[1])
        end_week = int(available_weeks[-1].split('-')[1])
        target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                        + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

        prd_total = []

        for target_idx in range(90-1-look_back+8):
            if mode == 'A':
                enc_outs, news_states, sns_states, trend_states = me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                                            [x_total[1][target_idx:target_idx+1],
                                                                                             x_total[2][target_idx:target_idx+1],
                                                                                             x_total[3][target_idx:target_idx+1]]])

                dec_input = [x_total[4][target_idx, 0, :].reshape(1, 1, x_total[4].shape[2]),
                             x_total[5][target_idx, 0, :].reshape(1, 1, x_total[5].shape[2]),
                             x_total[6][target_idx, 0, :].reshape(1, 1, x_total[6].shape[2])]
                tmp = []
                for ahead in range(look_ahead):
                    prd, news_states, sns_states, trend_states = me.decoder_model.predict(dec_input + enc_outs 
                                                                      + [news_states, sns_states, trend_states])
                    prd_val = prd[0, 0, 0]
                    if prd_val > 0:
                        tmp.append([prd_val])
                    else:
                        tmp.append([np.random.uniform(1e-6, 1e-5)])

                    if ahead < look_ahead - 1:
                        dec_input = [x_total[4][target_idx, ahead+1, :].reshape(1, 1, x_total[4].shape[2]),
                             x_total[5][target_idx, ahead+1, :].reshape(1, 1, x_total[5].shape[2]),
                             x_total[6][target_idx, ahead+1, :].reshape(1, 1, x_total[6].shape[2])]
                        dec_input[0][0, 0, -1] = prd_val
                        dec_input[1][0, 0, -1] = prd_val
                        dec_input[2][0, 0, -1] = prd_val
            else:
                enc_out, enc_h, enc_c = test_me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                      x_total[1][target_idx:target_idx+1]])

                dec_input = x_total[2][target_idx, 0, :].reshape(1, 1, x_total[2].shape[2])
                tmp = []
                for ahead in range(look_ahead):
                    prd, dec_h, dec_c = test_me.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
                    prd_val = prd[0, 0, 0]
                    if prd_val > 0:
                        tmp.append([prd_val])
                    else:
                        tmp.append([np.random.uniform(1e-6, 1e-5)])

                    if ahead < look_ahead - 1:
                        dec_input = x_total[2][target_idx, ahead+1, :].reshape(1, 1, x_total[2].shape[2])
                        dec_input[0, 0, -1] = prd_val
                    enc_h, enc_c = dec_h, dec_c
            prd_total.append(tmp)
        prd_total = np.array(prd_total)
        KR_prd_by_model[mode_name] = prd_total

        show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
        prd_train = prd_total[:val_size]
        prd_val = prd_total[val_size:test_size]
        prd_test = prd_total[test_size:]

        rmse_total, mape_total, corr_total = Evaluate.evaluate(target_truth, 
                                                               prd_total, look_ahead)
        rmse_train, mape_train, corr_train = Evaluate.evaluate(target_truth[:val_size+look_ahead], 
                                                               prd_train, look_ahead)
        rmse_val, mape_val, corr_val = Evaluate.evaluate(target_truth[val_size:test_size+look_ahead],
                                                         prd_val, look_ahead)
        rmse_test, mape_test, corr_test = Evaluate.evaluate(target_truth[test_size:],
                                                            prd_test, look_ahead)

        for ahead in range(look_ahead):
            ah = str(ahead+1)
            KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
            KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
            KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

            KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
            KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
            KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

            KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
            KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
            KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

            KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
            KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
            KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
#     print(KR_perform_by_model)
    KR_perform_by_coef[threshold]['perform'] = KR_perform_by_model.copy()
    KR_perform_by_coef[threshold]['predict'] = KR_prd_by_model.copy()

In [ ]:
with open('./data/v2/KR_perform_by_coef.pkl', 'wb') as f:
    pickle.dump(KR_perform_by_coef, f)

In [ ]:
KR_perform_by_model = {}

look_back = 7

for threshold in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    for mode in ['N', 'S', 'T']:
        mode_name = 'ME-' + mode
        KR_perform_by_model[mode_name] = {}
        
        available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
        start_week = int(available_weeks[0].split('-')[1])
        end_week = int(available_weeks[-1].split('-')[1])
        
        target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                        + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

        rmse_total, mape_total, corr_total = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1)), 
                                                               np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_coef[threshold]['predict'][mode_name]]),
                                                               look_ahead)
        rmse_train, mape_train, corr_train = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[:val_size+look_ahead], 
                                                               np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_coef[threshold]['predict'][mode_name]])[:val_size],
                                                               look_ahead)
        rmse_val, mape_val, corr_val = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[val_size:test_size+look_ahead],
                                                         np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_coef[threshold]['predict'][mode_name]])[val_size:test_size],
                                                         look_ahead)
        rmse_test, mape_test, corr_test = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[test_size:],
                                                            np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_coef[threshold]['predict'][mode_name]])[test_size:],
                                                            look_ahead)

        for ahead in range(look_ahead):
            ah = str(ahead+1)
            KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
            KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
            KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

            KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
            KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
            KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

            KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
            KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
            KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

            KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
            KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
            KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
    KR_perform_by_coef[threshold]['perform'] = KR_perform_by_model.copy()

In [ ]:
coef_compare = {'N': {}, 'S': {}, 'T': {}, 'mean': {}}

for ev in ['rmse', 'mape', 'corr']:
    coef_compare['N'][ev] = []
    coef_compare['S'][ev] = []
    coef_compare['T'][ev] = []
    coef_compare['mean'][ev] = []
    for coef_threshold in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
        tmp2 = pd.DataFrame(KR_perform_by_coef[coef_threshold]['perform']).T.loc[['ME-N', 'ME-S', 'ME-T']]
        n, s, t = tmp2[[c for c in tmp2.columns if 'total' in c and ev in c]].mean(axis=1).values
        
        coef_compare['N'][ev].append(n)
        coef_compare['S'][ev].append(s)
        coef_compare['T'][ev].append(t)
        coef_compare['mean'][ev].append(tmp2[[c for c in tmp2.columns if 'total' in c and ev in c]].mean(axis=1).mean())
        
for mode in ['N', 'S', 'T', 'mean']:
    coef_compare[mode] = pd.DataFrame(coef_compare[mode], index=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
    coef_compare[mode]['rmse(improve)'] = coef_compare[mode]['rmse'].apply(lambda x: (coef_compare[mode]['rmse'].max() - x) / coef_compare[mode]['rmse'].max())
    coef_compare[mode]['mape(improve)'] = coef_compare[mode]['mape'].apply(lambda x: (coef_compare[mode]['mape'].max() - x) / coef_compare[mode]['mape'].max())
    coef_compare[mode]['corr(improve)'] = coef_compare[mode]['corr'].apply(lambda x: (x - coef_compare[mode]['corr'].min()) / coef_compare[mode]['corr'].min())
    coef_compare[mode]['improve(mean)'] = coef_compare[mode][['rmse(improve)', 'mape(improve)', 'corr(improve)']].mean(axis=1)
    coef_compare[mode].to_csv('./data/v2/KR_perform_by_coef_' + mode + '.csv', encoding='utf8')

In [ ]:
coef_compare['mean'].round(3)

In [ ]:
# KR_N,S,T MEDIF , max_lag lag threshold param check

KR_perform_by_lag = {}
KR_perform_by_model = {}
KR_prd_by_model = {}

look_back = 7

mode = 'T'

for threshold in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    
    KR_news_tmp_max_lag = {}
    lag_max = KR_news_lag.max()
    lag_max = lag_max[lag_max > 0.5]
    for word in lag_max.index:
        lag = KR_news_lag[word].argmax()
        val = KR_news_lag.loc[lag, word]
        if lag >= threshold:
            KR_news_tmp_max_lag[word] = (lag, val)
    KR_news_tmp_max_lag = pd.DataFrame(KR_news_tmp_max_lag, index=['max_lag', 'coef'])
    
    KR_sns_tmp_max_lag = {}
    lag_max = KR_sns_lag.max()
    lag_max = lag_max[lag_max > 0.5]
    for word in lag_max.index:
        lag = KR_sns_lag[word].argmax()
        val = KR_sns_lag.loc[lag, word]
        if lag >= threshold:
            KR_sns_tmp_max_lag[word] = (lag, val)
    KR_sns_tmp_max_lag = pd.DataFrame(KR_sns_tmp_max_lag, index=['max_lag', 'coef'])
    
    KR_trends_tmp_max_lag = {}
    lag_max = KR_trends_lag.max()
    lag_max = lag_max[lag_max > 0.5]
    for word in lag_max.index:
        lag = KR_trends_lag[word].argmax()
        val = KR_trends_lag.loc[lag, word]
        if lag >= threshold:
            KR_trends_tmp_max_lag[word] = (lag, val)
    KR_trends_tmp_max_lag = pd.DataFrame(KR_trends_tmp_max_lag, index=['max_lag', 'coef'])
    
    print('='*5, 'threshold:', threshold, '='*5)
    KR_perform_by_lag[threshold] = {}
    for mode in ['N', 'S', 'T', 'A']:
        mode_name = 'ME-' + mode
        KR_perform_by_model[mode_name] = {}
        print('mode:', mode_name)

        clear_session()

        news_word_num = len(KR_news_tmp_max_lag.columns)
        sns_word_num = len(KR_sns_tmp_max_lag.columns)
        trend_word_num = len(KR_trends_tmp_max_lag.columns)

        me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                              past_year_num=6,
                              news_word_num=news_word_num,
                              sns_word_num=sns_word_num,
                              trend_word_num=trend_word_num,
                              mode=mode)

        available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
        x, y, val_size, test_size = me.generate_data(KCDC_norm, available_weeks,
                                                     look_back, look_ahead,
                                                     test_split_size=test_split_size,
                                                     val_split_size=val_split_size,
                                                     news_data=KR_news_norm,
                                                     news_lag=KR_news_tmp_max_lag,
                                                     sns_data=KR_sns_norm,
                                                     sns_lag=KR_sns_tmp_max_lag,
                                                     trends_data=KR_trends_norm,
                                                     trends_lag=KR_trends_tmp_max_lag)

        x_train, x_val, x_test, x_total = x
        y_train, y_val, y_test = y

        print('val size:', val_size, 'test_size:', test_size)

        cp = ModelCheckpoint('./models/v2/me_{epoch}.h5')

        history = me.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                          callbacks=[cp], verbose=0)

        plt.plot(history.history['loss'], label='train_loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.legend()
        plt.show()

        min_epoch = 100
        target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                        min(history.history['val_loss'][min_epoch:]))
        best_path = './models/v2/KR_me_' + mode + '_lb' + str(look_back) + '_best.h5'
        print(target_epoch, best_path)
        shutil.copy('./models/v2/me_' + str(target_epoch) + '.h5', best_path)


        test_me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                                  past_year_num=past_year_num,
                                  news_word_num=news_word_num,
                                  sns_word_num=sns_word_num,
                                  trend_word_num=trend_word_num,
                                  mode=mode)
        test_me.load_model(best_path)

        start_week = int(available_weeks[0].split('-')[1])
        end_week = int(available_weeks[-1].split('-')[1])
        target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                        + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

        prd_total = []

        for target_idx in range(90-1-look_back+8):
            if mode == 'A':
                enc_outs, news_states, sns_states, trend_states = me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                                            [x_total[1][target_idx:target_idx+1],
                                                                                             x_total[2][target_idx:target_idx+1],
                                                                                             x_total[3][target_idx:target_idx+1]]])

                dec_input = [x_total[4][target_idx, 0, :].reshape(1, 1, x_total[4].shape[2]),
                             x_total[5][target_idx, 0, :].reshape(1, 1, x_total[5].shape[2]),
                             x_total[6][target_idx, 0, :].reshape(1, 1, x_total[6].shape[2])]
                tmp = []
                for ahead in range(look_ahead):
                    prd, news_states, sns_states, trend_states = me.decoder_model.predict(dec_input + enc_outs 
                                                                      + [news_states, sns_states, trend_states])
                    prd_val = prd[0, 0, 0]
                    if prd_val > 0:
                        tmp.append([prd_val])
                    else:
                        tmp.append([np.random.uniform(1e-6, 1e-5)])

                    if ahead < look_ahead - 1:
                        dec_input = [x_total[4][target_idx, ahead+1, :].reshape(1, 1, x_total[4].shape[2]),
                             x_total[5][target_idx, ahead+1, :].reshape(1, 1, x_total[5].shape[2]),
                             x_total[6][target_idx, ahead+1, :].reshape(1, 1, x_total[6].shape[2])]
                        dec_input[0][0, 0, -1] = prd_val
                        dec_input[1][0, 0, -1] = prd_val
                        dec_input[2][0, 0, -1] = prd_val
            else:
                enc_out, enc_h, enc_c = test_me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                      x_total[1][target_idx:target_idx+1]])

                dec_input = x_total[2][target_idx, 0, :].reshape(1, 1, x_total[2].shape[2])
                tmp = []
                for ahead in range(look_ahead):
                    prd, dec_h, dec_c = test_me.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
                    prd_val = prd[0, 0, 0]
                    if prd_val > 0:
                        tmp.append([prd_val])
                    else:
                        tmp.append([np.random.uniform(1e-6, 1e-5)])

                    if ahead < look_ahead - 1:
                        dec_input = x_total[2][target_idx, ahead+1, :].reshape(1, 1, x_total[2].shape[2])
                        dec_input[0, 0, -1] = prd_val
                    enc_h, enc_c = dec_h, dec_c
            prd_total.append(tmp)
        prd_total = np.array(prd_total)
        KR_prd_by_model[mode_name] = prd_total

        show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
        prd_train = prd_total[:val_size]
        prd_val = prd_total[val_size:test_size]
        prd_test = prd_total[test_size:]

        rmse_total, mape_total, corr_total = Evaluate.evaluate(target_truth, 
                                                               prd_total, look_ahead)
        rmse_train, mape_train, corr_train = Evaluate.evaluate(target_truth[:val_size+look_ahead], 
                                                               prd_train, look_ahead)
        rmse_val, mape_val, corr_val = Evaluate.evaluate(target_truth[val_size:test_size+look_ahead],
                                                         prd_val, look_ahead)
        rmse_test, mape_test, corr_test = Evaluate.evaluate(target_truth[test_size:],
                                                            prd_test, look_ahead)

        for ahead in range(look_ahead):
            ah = str(ahead+1)
            KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
            KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
            KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

            KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
            KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
            KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

            KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
            KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
            KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

            KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
            KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
            KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
#     print(KR_perform_by_model)
    KR_perform_by_lag[threshold]['perform'] = KR_perform_by_model.copy()
    KR_perform_by_lag[threshold]['predict'] = KR_prd_by_model.copy()

In [ ]:
with open('./data/v2/KR_perform_by_lag.pkl', 'wb') as f:
    pickle.dump(KR_perform_by_lag, f)

In [ ]:
KR_perform_by_model = {}

look_back = 7

for threshold in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    for mode in ['N', 'S', 'T']:
        mode_name = 'ME-' + mode
        KR_perform_by_model[mode_name] = {}
        
        available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
        start_week = int(available_weeks[0].split('-')[1])
        end_week = int(available_weeks[-1].split('-')[1])
        
        target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                        + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

        rmse_total, mape_total, corr_total = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1)), 
                                                               np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_lag[threshold]['predict'][mode_name]]),
                                                               look_ahead)
        rmse_train, mape_train, corr_train = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[:val_size+look_ahead], 
                                                               np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_lag[threshold]['predict'][mode_name]])[:val_size],
                                                               look_ahead)
        rmse_val, mape_val, corr_val = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[val_size:test_size+look_ahead],
                                                         np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_lag[threshold]['predict'][mode_name]])[val_size:test_size],
                                                         look_ahead)
        rmse_test, mape_test, corr_test = Evaluate.evaluate(KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))[test_size:],
                                                            np.array([KCDC_scaler.inverse_transform(_) for _ in KR_perform_by_lag[threshold]['predict'][mode_name]])[test_size:],
                                                            look_ahead)

        for ahead in range(look_ahead):
            ah = str(ahead+1)
            KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
            KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
            KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

            KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
            KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
            KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

            KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
            KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
            KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

            KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
            KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
            KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
    KR_perform_by_lag[threshold]['perform'] = KR_perform_by_model.copy()

In [ ]:
lag_compare = {'N': {}, 'S': {}, 'T': {}, 'mean': {}}

for ev in ['rmse', 'mape', 'corr']:
    lag_compare['N'][ev] = []
    lag_compare['S'][ev] = []
    lag_compare['T'][ev] = []
    lag_compare['mean'][ev] = []
    for lag_threshold in np.arange(0, 11):
        tmp2 = pd.DataFrame(KR_perform_by_lag[lag_threshold]['perform']).T.loc[['ME-N', 'ME-S', 'ME-T']]
        n, s, t = tmp2[[c for c in tmp2.columns if 'total' in c and ev in c]].mean(axis=1).values
        
        lag_compare['N'][ev].append(n)
        lag_compare['S'][ev].append(s)
        lag_compare['T'][ev].append(t)
        lag_compare['mean'][ev].append(tmp2[[c for c in tmp2.columns if 'total' in c and ev in c]].mean(axis=1).mean())
        
for mode in ['N', 'S', 'T', 'mean']:
    lag_compare[mode] = pd.DataFrame(lag_compare[mode], index=np.arange(0, 11))
    lag_compare[mode]['rmse(improve)'] = lag_compare[mode]['rmse'].apply(lambda x: (lag_compare[mode]['rmse'].max() - x) / lag_compare[mode]['rmse'].max())
    lag_compare[mode]['mape(improve)'] = lag_compare[mode]['mape'].apply(lambda x: (lag_compare[mode]['mape'].max() - x) / lag_compare[mode]['mape'].max())
    lag_compare[mode]['corr(improve)'] = lag_compare[mode]['corr'].apply(lambda x: (x - lag_compare[mode]['corr'].min()) / lag_compare[mode]['corr'].min())
    lag_compare[mode]['improve(mean)'] = lag_compare[mode][['rmse(improve)', 'mape(improve)', 'corr(improve)']].mean(axis=1)
    lag_compare[mode].to_csv('./data/v2/KR_perform_by_lag_' + mode + '.csv', encoding='utf8')

In [ ]:
lag_compare['mean'].round(3)

In [ ]:
# KR_N,S,T MEDIF 적용
epochs = 200
is_sg = False
sg_path = '_2y' # if is_sg else ''

# KR_perform_by_model = {}
# KR_prd_by_model = {}

mode_num_words = {'N': 10, 'S': 6, 'T': 9}
#     perform_by_num_words['KR'][i] = {}
for mode in ['S']:
    i = mode_num_words[mode]
    mode_name = 'ME-' + mode
    mode_name = mode_name+'(2Y)' # if is_sg else mode_name
    
    KR_perform_by_model[mode_name] = {}
    print('mode:', mode_name)

    clear_session()

    news_target_max_lag = KR_news_max_lag.T.sort_values(by='coef', ascending=False).T
    sns_target_max_lag = KR_sns_max_lag.T.sort_values(by='coef', ascending=False).head(6).T
    trends_target_max_lag = KR_trends_max_lag.T.sort_values(by='coef', ascending=False).head(9).T

    if mode == 'N':
        print(i, 'news', news_target_max_lag.columns)
    elif mode == 'S':
        print(i, 'sns', sns_target_max_lag.columns)
    else:
        print(i, 'trends', trends_target_max_lag.columns)

    news_word_num = len(news_target_max_lag.columns)
    news_word_num = news_word_num+1 if is_sg else news_word_num
    sns_word_num = len(sns_target_max_lag.columns)
    sns_word_num = sns_word_num+1 if is_sg else sns_word_num
    trend_word_num = len(trends_target_max_lag.columns)
    trend_word_num = trend_word_num+1 if is_sg else trend_word_num

    me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                          past_year_num=4,
                          news_word_num=news_word_num,
                          sns_word_num=sns_word_num,
                          trend_word_num=trend_word_num,
                          mode=mode,
                          is_sg=is_sg)

    available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]
    x, y, val_size, test_size = me.generate_data(KCDC_norm, available_weeks,
                                                 look_back, look_ahead,
                                                 test_split_size=test_split_size,
                                                 val_split_size=val_split_size,
                                                 news_data=KR_news_norm,
                                                 news_lag=news_target_max_lag,
                                                 sns_data=KR_sns_norm,
                                                 sns_lag=sns_target_max_lag,
                                                 trends_data=KR_trends_norm,
                                                 trends_lag=trends_target_max_lag)

    x_train, x_val, x_test, x_total = x
    y_train, y_val, y_test = y

    print('val size:', val_size, 'test_size:', test_size)

    cp = ModelCheckpoint('./models/v2/me_{epoch}.h5')

    history = me.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                      callbacks=[cp], verbose=0)

    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()

    min_epoch = 100
    target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                    min(history.history['val_loss'][min_epoch:]))
    best_path = './models/v2/KR_me_' + mode + '_lb' + str(look_back) + sg_path + '_best.h5'
    print(target_epoch, best_path)
    shutil.copy('./models/v2/me_' + str(target_epoch) + '.h5', best_path)


    test_me = Multi_Encoder_v2(look_back=look_back, look_ahead=look_ahead,
                              past_year_num=4,
                              news_word_num=news_word_num,
                              sns_word_num=sns_word_num,
                              trend_word_num=trend_word_num,
                              mode=mode,
                              is_sg=is_sg)
    test_me.load_model(best_path)

    start_week = int(available_weeks[0].split('-')[1])
    end_week = int(available_weeks[-1].split('-')[1])
    target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                    + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

    prd_total = []

    for target_idx in range(90-1):
        if mode == 'A':
            enc_outs, news_states, sns_states, trend_states = me.encoder_model.predict([x_total[0][target_idx:target_idx+1],
                                                                                        [x_total[1][target_idx:target_idx+1],
                                                                                         x_total[2][target_idx:target_idx+1],
                                                                                         x_total[3][target_idx:target_idx+1]]])

            dec_input = [x_total[4][target_idx, 0, :].reshape(1, 1, x_total[4].shape[2]),
                         x_total[5][target_idx, 0, :].reshape(1, 1, x_total[5].shape[2]),
                         x_total[6][target_idx, 0, :].reshape(1, 1, x_total[6].shape[2])]
            tmp = []
            for ahead in range(look_ahead):
                prd, news_states, sns_states, trend_states = me.decoder_model.predict(dec_input + enc_outs 
                                                                  + [news_states, sns_states, trend_states])
                prd_val = prd[0, 0, 0]
                if prd_val > 0:
                    tmp.append([prd_val])
                else:
                    tmp.append([np.random.uniform(1e-6, 1e-5)])

                if ahead < look_ahead - 1:
                    dec_input = [x_total[4][target_idx, ahead+1, :].reshape(1, 1, x_total[4].shape[2]),
                         x_total[5][target_idx, ahead+1, :].reshape(1, 1, x_total[5].shape[2]),
                         x_total[6][target_idx, ahead+1, :].reshape(1, 1, x_total[6].shape[2])]
                    dec_input[0][0, 0, -1] = prd_val
                    dec_input[1][0, 0, -1] = prd_val
                    dec_input[2][0, 0, -1] = prd_val
        else:
            if is_sg:
              enc_inputs = [x_total[0][target_idx:target_idx+1]]
              dec_input = x_total[1][target_idx, 0, :].reshape(1, 1, x_total[1].shape[2])
            else:
              enc_inputs = [x_total[0][target_idx:target_idx+1],
                            x_total[1][target_idx:target_idx+1]]
              dec_input = x_total[2][target_idx, 0, :].reshape(1, 1, x_total[2].shape[2])
            enc_out, enc_h, enc_c = test_me.encoder_model.predict(enc_inputs)

            tmp = []
            for ahead in range(look_ahead):
                prd, dec_h, dec_c = test_me.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
                prd_val = prd[0, 0, 0]
                if prd_val > 0:
                    tmp.append([prd_val])
                else:
                    tmp.append([np.random.uniform(1e-6, 1e-5)])

                if ahead < look_ahead - 1:
                    if is_sg:
                      dec_input = x_total[1][target_idx, ahead+1, :].reshape(1, 1, x_total[1].shape[2])
                    else:
                      dec_input = x_total[2][target_idx, ahead+1, :].reshape(1, 1, x_total[2].shape[2])
                    dec_input[0, 0, -1] = prd_val
                enc_h, enc_c = dec_h, dec_c
        prd_total.append(tmp)
    prd_total = np.array(prd_total)
    KR_prd_by_model[mode_name] = prd_total

    show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
    prd_train = prd_total[:val_size]
    prd_val = prd_total[val_size:test_size]
    prd_test = prd_total[test_size:]

    inverse_target_truth = KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
    inverse_prd_total = np.array([KCDC_scaler.inverse_transform(_) for _ in prd_total])


    rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                           inverse_prd_total, look_ahead)
    rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                           inverse_prd_total, look_ahead)
    rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                           inverse_prd_total[:val_size], look_ahead)
    rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                     inverse_prd_total[val_size:test_size], look_ahead)
    rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                        inverse_prd_total[test_size:], look_ahead)

    for ahead in range(look_ahead):
        ah = str(ahead+1)
        KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
        KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
        KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

        KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
        KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
        KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

        KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
        KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
        KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

        KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
        KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
        KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]
    perform_by_num_words['KR'][i]['perform'][mode_name] = KR_perform_by_model[mode_name].copy()
    perform_by_num_words['KR'][i]['prd'][mode_name] = prd_total

In [ ]:
pd.concat([KR_perform, pd.DataFrame(KR_perform_by_model).T])[
  ['total_%s_%s' % (ev, i) for i in [5, 8, 10] for ev in ['rmse', 'mape', 'corr']]].round(2)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(9, 3))
ev = 'rmse'

perform_vals = pd.DataFrame(perform_by_num_words['US'][1]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-T(US)'].values
axs[0].plot(perform_vals)
perform_vals = pd.DataFrame(perform_by_num_words['US'][1]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-T(US)(SG)'].values
axs[0].plot(perform_vals, ls='--')

perform_vals = pd.DataFrame(perform_by_num_words['AU'][1]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-T(AU)'].values
axs[1].plot(perform_vals)
perform_vals = pd.DataFrame(perform_by_num_words['AU'][1]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-T(AU)(SG)'].values
axs[1].plot(perform_vals, ls='--')

perform_vals = pd.DataFrame(perform_by_num_words['KR'][10]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-N'].values
axs[2].plot(perform_vals)
perform_vals = pd.DataFrame(perform_by_num_words['KR'][10]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-N(SG)'].values
axs[2].plot(perform_vals, ls='--')

perform_vals = pd.DataFrame(perform_by_num_words['KR'][6]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-S'].values
axs[2].plot(perform_vals)
perform_vals = pd.DataFrame(perform_by_num_words['KR'][6]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-S(SG)'].values
axs[2].plot(perform_vals, ls='--')

perform_vals = pd.DataFrame(perform_by_num_words['KR'][9]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-T'].values
axs[2].plot(perform_vals)
perform_vals = pd.DataFrame(perform_by_num_words['KR'][9]['perform']).T[
  ['total_%s_%s' % (ev, i) for i in range(1, 11)]
].T['ME-T(SG)'].values
axs[2].plot(perform_vals, ls='--')



plt.show()

In [ ]:
# 3개 국가의 1, 5, 10주 예측 그래프

markers = ['.', 's', 'D', 'v', 'h']
colors = ['tab:brown', 'tab:purple', 'tab:pink', 'tab:olive', 'tab:blue']
line_styles = ['--', '-.', '--', '-', '-']
x_labels = ['week\n(a) ahead-', 'week\n(b) ahead-', 'week\n(c) ahead-']

fig, axs = plt.subplots(3, 3, figsize=(15, 15))

for col, ev in enumerate(['rmse', 'mape', 'corr']):
    target_df = US_perform[[c for c in US_perform if 'total' in c and ev in c]]
    for i, m in enumerate(['Basic-LSTM(US)', 'DEFSI(US)', 'Seq2Seq(US)', 'ME-T(US)(SG)', 'ME-T(US)']):
        perform_vals = (target_df.T[m].values if 'ME-' not in m 
                        else pd.DataFrame(perform_by_num_words['US'][1]['perform']).T[
                            ['total_%s_%s' % (ev, i) for i in range(1, 11)]
                        ].T[m].values.reshape(-1))
        axs[0, col].plot(perform_vals, label='MEDIF-' + m[3:], marker=markers[i], c=colors[i],
                ls=line_styles[i])
        axs[0, col].set_ylabel(ev.upper())
        axs[0, col].set_xlabel('Prediction-period (weeks)')
        axs[0, col].set_xticks(range(1, 11, 2))
        axs[0, col].set_xticklabels(range(2, 12, 2))
        axs[0, col].grid(True, ls=':')
axs[0, 1].set_xlabel('Prediction-period (weeks)\n(a) United States')
        
for col, ev in enumerate(['rmse', 'mape', 'corr']):
    target_df = AU_perform[[c for c in AU_perform if 'total' in c and ev in c]]
    for i, m in enumerate(['Basic-LSTM(AU)', 'DEFSI(AU)', 'Seq2Seq(AU)', 'ME-T(AU)(SG)', 'ME-T(AU)']):
        perform_vals = (target_df.T[m].values if 'ME-' not in m 
                        else pd.DataFrame(perform_by_num_words['AU'][1]['perform']).T[
                            ['total_%s_%s' % (ev, i) for i in range(1, 11)]
                        ].T[m].values.reshape(-1))
        
        axs[1, col].plot(perform_vals, label='MEDIF-' + m[3:], marker=markers[i], c=colors[i],
                ls=line_styles[i])
        axs[1, col].set_ylabel(ev.upper())
        axs[1, col].set_xlabel('Prediction-period (weeks)')
        axs[1, col].set_xticks(range(1, 11, 2))
        axs[1, col].set_xticklabels(range(2, 12, 2))
        axs[1, col].grid(True, ls=':')
axs[1, 1].set_xlabel('Prediction-period (weeks)\n(b) Australia')
        
markers = ['.', 'v', 's', 'p', 'D', 'h', '*']
colors = ['tab:brown', 'tab:purple', 'tab:pink', 'tab:red', 'tab:green', 'tab:blue']
line_styles = ['--', '-.', '--', '-', '-', '-']
for col, ev in enumerate(['rmse', 'mape', 'corr']):
    target_df = KR_perform[[c for c in KR_perform if 'total' in c and ev in c]]
    for i, m in enumerate(['Basic-LSTM', 'DEFSI', 'Seq2Seq', 'ME-N', 'ME-S', 'ME-T']):
        if 'ME-' not in m:
            perform_vals = target_df.T[m].values
        else:
            if m.split('-')[1] == 'N':
                perform_vals = target_df.T[m].values
            elif m.split('-')[1] == 'S':
                perform_vals = pd.DataFrame(perform_by_num_words['KR'][6]['perform']).T[
                                ['total_%s_%s' % (ev, i) for i in range(1, 11)]
                                ].T[m].values
            else:
                perform_vals = pd.DataFrame(perform_by_num_words['KR'][9]['perform']).T[
                                ['total_%s_%s' % (ev, i) for i in range(1, 11)]
                                ].T[m].values
                
                
#         perform_vals = (target_df.T[m].values if 'ME-' not in m 
#                         else pd.DataFrame(perform_by_num_words['KR'][9]['perform']).T[
#                             ['total_%s_%s' % (ev, i) for i in range(1, 11)]
#                         ].T[m].values)
        
        axs[2, col].plot(perform_vals, marker=markers[i], c=colors[i],
                ls=line_styles[i], label=m)
        axs[2, col].set_ylabel(ev.upper())
        axs[2, col].set_xlabel('Prediction-period (weeks)')
        axs[2, col].set_xticks(range(1, 11, 2))
        axs[2, col].set_xticklabels(range(2, 12, 2))
        axs[2, col].grid(True, ls=':')
axs[2, 1].set_xlabel('Prediction-period (weeks)\n(c) Korea')

plt.legend(loc='upper center', bbox_to_anchor=(-0.7, -0.25), ncol=6,
          labels=['LSTM', 'DEFSI', 'STS-ATT',
                  'Proposed(N)', 'Proposed(S)', 'Proposed(Q)'])
# plt.savefig('./image/v3/All_perform_apply_num_words_down_dpi.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
with open('./data/v2/perform_by_num_words2.pkl', 'wb') as f:
    pickle.dump(perform_by_num_words, f)

In [ ]:
## Basic_LSTM

epochs = 70

# KR_perform_by_model = {}
mode_name = 'Basic-LSTM'
KR_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

basic_lstm = Basic_LSTM(look_back=look_back, look_ahead=look_ahead)
available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = basic_lstm.generate_data(KCDC_norm, available_weeks,
                                                  look_back, 1, 
                                                  test_split_size=test_split_size,
                                                  val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/basic_lstm_{epoch}.h5')

history = basic_lstm.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 30
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))
best_path = './models/v2/KR_basic_lstm_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/basic_lstm_' + str(target_epoch) + '.h5', best_path)

test_basic_lstm = Basic_LSTM(look_back=look_back, look_ahead=look_ahead)
test_basic_lstm.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())


prd_total = []
for target_idx in range(90-1):    
    tmp = []
    for i in range(look_ahead):
        if i == 0:
            left_shift = np.array([x_total[0][target_idx]])
        else:
            left_shift = np.roll(left_shift, -1)
            left_shift[:, -1, :] = tmp[-1]
        prd = test_basic_lstm.model.predict(left_shift)[0, 0]
        prd = np.random.uniform(1e-6, 1e-5) if prd < 0 else prd
        prd = 1.0 - np.random.uniform(1e-6, 1e-5) if prd > 1 else prd
        tmp.append(prd)
#     print(left_shift)
#     print(tmp)
    prd_total.append(tmp)
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
prd_total = np.array(prd_total)
prd_total = prd_total.reshape((prd_total.shape[0], prd_total.shape[1], 1))
KR_prd_by_model[mode_name] = prd_total

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([KCDC_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
## DEFSI

test_split_size = 0.65
val_split_size = 0.7
epochs = 300

# KR_perform_by_model = {}
mode_name = 'DEFSI'
KR_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

defsi = DEFSI(look_back=look_back, look_ahead=1)
available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = defsi.generate_data(KCDC_norm, available_weeks,
                                              look_back, 1, 
                                              test_split_size=test_split_size,
                                              val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/defsi_{epoch}.h5')

history = defsi.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 100
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))
best_path = './models/v2/KR_defsi_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/defsi_' + str(target_epoch) + '.h5', best_path)

test_defsi = DEFSI(look_back=look_back, look_ahead=1)
test_defsi.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

prd_total = []
for target_idx in range(90-1):    
    tmp = []
    for i in range(look_ahead):
        if i == 0:
            left_shift = np.array([x_total[0][target_idx]])
        else:
            left_shift = np.roll(left_shift, -1)
            left_shift[:, -1, :] = tmp[-1]
        prd = test_defsi.model.predict([left_shift, np.array([x_total[1][target_idx+1]])])[0, 0]
        prd = np.random.uniform(1e-6, 1e-5) if prd < 0 else prd
        tmp.append(prd)
#     print(left_shift)
#     print(tmp)
    prd_total.append(tmp)
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
prd_total = np.array(prd_total)
prd_total = prd_total.reshape((prd_total.shape[0], prd_total.shape[1], 1))
KR_prd_by_model[mode_name] = prd_total

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([KCDC_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
## Seq2Seq

# KR_perform_by_model = {}
mode_name = 'Seq2Seq'
KR_perform_by_model[mode_name] = {}
print(mode_name)
clear_session()

seq2seq = Seq2Seq(look_back=look_back, look_ahead=look_ahead)

available_weeks = KR_trends_norm['weeks'].values[collect_err+max_lag+look_back:-look_ahead+1]

x, y, val_size, test_size = seq2seq.generate_data(KCDC_norm, available_weeks,
                                                  look_back, look_ahead, 
                                                  test_split_size=test_split_size,
                                                  val_split_size=val_split_size)
x_train, x_val, x_test, x_total = x
y_train, y_val, y_test = y

cp = ModelCheckpoint('./models/v2/seq2seq_{epoch}.h5')

history = seq2seq.fit(x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size,
                  callbacks=[cp], verbose=0)

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

min_epoch = 200
target_epoch = min_epoch + history.history['val_loss'][min_epoch:].index(
                                min(history.history['val_loss'][min_epoch:]))

best_path = './models/v2/KR_seq2seq_best.h5'
print(target_epoch, best_path)
shutil.copy('./models/v2/seq2seq_' + str(target_epoch) + '.h5', best_path)

test_seq2seq = Seq2Seq(look_back=look_back, look_ahead=look_ahead)
test_seq2seq.load_model(best_path)

start_week = int(available_weeks[0].split('-')[1])
end_week = int(available_weeks[-1].split('-')[1])
target_truth = (KCDC_norm.loc[start_week:52, '2018'].tolist() + KCDC_norm.loc[:52, '2019'].tolist()
                + KCDC_norm.loc[:end_week+look_ahead-1, '2020'].tolist())

prd_total = []

for target_idx in range(90-1):
    enc_out, enc_h, enc_c = test_seq2seq.encoder_model.predict(np.array([x_total[0][target_idx]]))

    dec_input = x_total[1][target_idx][0].reshape(-1, 1)

    tmp = []
    for ahead in range(look_ahead):
        prd, dec_h, dec_c = test_seq2seq.decoder_model.predict([dec_input] + [enc_out, enc_h, enc_c])
        prd_val = prd[0, 0]
        if prd[0, 0] > 0:
            tmp.append(prd_val)
        else:
            tmp.append([np.random.uniform(1e-4, 1e-3)])

        dec_input = prd[0, 0]
        enc_h, enc_c = dec_h, dec_c
    prd_total.append(tmp)
prd_total = np.array(prd_total)
show_graph(target_truth, prd_total, look_ahead, val_size, test_size)
prd_total = np.array(prd_total)
KR_prd_by_model[mode_name] = prd_total

prd_train = prd_total[:val_size]
prd_val = prd_total[val_size:test_size]
prd_test = prd_total[test_size:]

inverse_target_truth = KCDC_scaler.inverse_transform(np.array(target_truth).reshape(-1, 1))
inverse_prd_total = np.array([KCDC_scaler.inverse_transform(_) for _ in prd_total])

rmse_total, mape_total, corr_total = Evaluate.evaluate(inverse_target_truth, 
                                                       inverse_prd_total, look_ahead)
rmse_train, mape_train, corr_train = Evaluate.evaluate(inverse_target_truth[:val_size+look_ahead], 
                                                       inverse_prd_total[:val_size], look_ahead)
rmse_val, mape_val, corr_val = Evaluate.evaluate(inverse_target_truth[val_size:test_size+look_ahead],
                                                 inverse_prd_total[val_size:test_size], look_ahead)
rmse_test, mape_test, corr_test = Evaluate.evaluate(inverse_target_truth[test_size:],
                                                    inverse_prd_total[test_size:], look_ahead)

for ahead in range(look_ahead):
    ah = str(ahead+1)
    KR_perform_by_model[mode_name]['total_rmse_' + ah] = rmse_total[ahead]
    KR_perform_by_model[mode_name]['total_mape_' + ah] = mape_total[ahead]
    KR_perform_by_model[mode_name]['total_corr_' + ah] = corr_total[ahead]

    KR_perform_by_model[mode_name]['train_rmse_' + ah] = rmse_train[ahead]
    KR_perform_by_model[mode_name]['train_mape_' + ah] = mape_train[ahead]
    KR_perform_by_model[mode_name]['train_corr_' + ah] = corr_train[ahead]

    KR_perform_by_model[mode_name]['val_rmse_' + ah] = rmse_val[ahead]
    KR_perform_by_model[mode_name]['val_mape_' + ah] = mape_val[ahead]
    KR_perform_by_model[mode_name]['val_corr_' + ah] = corr_val[ahead]

    KR_perform_by_model[mode_name]['test_rmse_' + ah] = rmse_test[ahead]
    KR_perform_by_model[mode_name]['test_mape_' + ah] = mape_test[ahead]
    KR_perform_by_model[mode_name]['test_corr_' + ah] = corr_test[ahead]

In [ ]:
KR_perform = pd.concat([KR_perform, pd.DataFrame(KR_perform_by_model).T])

In [ ]:
# KR_perform = pd.DataFrame(KR_perform_by_model).T
KR_perform.to_csv('./data/v2/KR_perform6.csv', encoding='utf8')
KR_perform

In [ ]:
# KR perform table

KR_perform_mini = KR_perform[['total_rmse_1', 'total_mape_1', 'total_corr_1', 
            'total_rmse_5', 'total_mape_5', 'total_corr_5', 
            'total_rmse_10', 'total_mape_10', 'total_corr_10', ]].loc[['Basic-LSTM', 'DEFSI', 'Seq2Seq', 'ME-N', 'ME-S', 'ME-T']].round(3)
KR_perform_mini

In [ ]:
with open('./data/v2/KR_predict4.pkl', 'wb') as f:
    pickle.dump(KR_prd_by_model, f)